# Models to detect Breaches

In [ ]:

#!/usr/bin/env python3
"""
Breach detection
This module handles:
- LSTM classifier (PyTorch with CUDA)
- GRU classifier (PyTorch with CUDA)
- FTA-GRU simplified classifier (PyTorch with CUDA)
- FTA-LSTM simplified classifier (PyTorch with CUDA)
- Temporal GNN classifier (timesteps-as-nodes; PyTorch with CUDA)
- IFCNN-TPP classifier (CNN + temporal encoder; PyTorch with CUDA)
- CEP3 classifier (dilated temporal convolution encoder; PyTorch with CUDA)
- GCN bipartite classifier (PyTorch with CUDA)
- GAT bipartite classifier (PyTorch with CUDA)

All models support:
- Baseline A: Per-platform models
- Model B: Cross-platform model 
GPU Acceleration:
- All PyTorch models use CUDA/MPS when available
- Automatic mixed precision (AMP) for faster training
"""


from __future__ import annotations

import os
import json
import time
import warnings
import traceback
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, roc_auc_score,
    average_precision_score, confusion_matrix
)
from sklearn.preprocessing import OneHotEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

# ---- Multiprocessing FD pressure fix (Linux) ----
import torch.multiprocessing as mp
try:
    mp.set_sharing_strategy("file_system")
except Exception:
    pass


# -------------------------
# Configuration
# -------------------------

BASE_FEATURES = [
    "sim_to_prev_passwords_max",
    "sim_to_prev_passwords_mean",
    "exact_password_reuse",
    "sim_to_prev_honeywords_max",
    "sim_to_prev_honeywords_mean",
    "honeyword_trigger",
    "current_hw_matches_prev_pw",
    "time_since_last_leak",
    "time_since_last_attack",
    "attack_sequence_index",
    "delta_similarity",
]

CROSS_FEATURES = [
    "num_platforms_seen",
    "platform_overlap_score",
    "cross_platform_reuse",
]

ID_COLS = ["Email", "Breach_Source", "AttackTime", "Timestamp"]


# -------------------------
# Utils
# -------------------------

def seed_everything(seed: int = 42) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

def save_json(obj: Any, path: str) -> None:
    with open(path, "w") as f:
        json.dump(obj, f, indent=2, default=str)

def load_json(path: str) -> Any:
    with open(path, "r") as f:
        return json.load(f)

def get_device() -> torch.device:
    """Get the best available device."""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using CUDA GPU: {torch.cuda.get_device_name(0)}")
        print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using Apple MPS GPU")
    else:
        device = torch.device("cpu")
        print("Using CPU")
    return device

def make_loader(dataset, batch_size, shuffle, device, setting: str):
    """
    SAFE DataLoader builder (prevents Errno 24 "Too many open files"):

    - We force num_workers=0 for ALL settings/models because you are running a grid
      (cross_platform + per_platform + multiple models) inside one long process.
    - persistent_workers is always False.
    """
    pin = (device.type == "cuda")
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=0,              # IMPORTANT: avoid FD exhaustion
        pin_memory=pin,
        persistent_workers=False,   # IMPORTANT: don't keep workers around
    )


# -------------------------
# Data Loading
# -------------------------

def load_cached(out_dir: str) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Load cached train/val/test splits from Stage 1."""
    paths = {
        "train": os.path.join(out_dir, "train.parquet"),
        "val": os.path.join(out_dir, "val.parquet"),
        "test": os.path.join(out_dir, "test.parquet"),
    }

    for key, path in paths.items():
        if not os.path.exists(path):
            raise FileNotFoundError(f"Missing {path}. Run Stage 1 first.")

    train = pd.read_parquet(paths["train"])
    val = pd.read_parquet(paths["val"])
    test = pd.read_parquet(paths["test"])

    for df in [train, val, test]:
        df["AttackTime"] = pd.to_datetime(df["AttackTime"], errors="coerce")
        df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

    return train, val, test


# -------------------------
# Design Matrices
# -------------------------

def build_design_matrices(
    train: pd.DataFrame,
    val: pd.DataFrame,
    test: pd.DataFrame,
    setting: str,
    platform_encoder: Optional[OneHotEncoder] = None,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, Optional[OneHotEncoder], Dict]:
    """
    Build feature matrices based on setting:
    - per_platform: base features only
    - cross_platform: base + cross features + one-hot platform
    """
    if setting not in {"per_platform", "pooled", "cross_platform"}:
        raise ValueError(f"Unknown setting={setting}")

    use_cross = (setting == "cross_platform")
    feat_cols = list(BASE_FEATURES) + (list(CROSS_FEATURES) if use_cross else [])

    def _num_df(d: pd.DataFrame) -> np.ndarray:
        return d[feat_cols].astype(float).values

    Xtr_num, Xva_num, Xte_num = _num_df(train), _num_df(val), _num_df(test)

    if setting == "cross_platform":
        if platform_encoder is None:
            platform_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
            platform_encoder.fit(train[["Breach_Source"]].astype(str))
        Ptr = platform_encoder.transform(train[["Breach_Source"]].astype(str))
        Pva = platform_encoder.transform(val[["Breach_Source"]].astype(str))
        Pte = platform_encoder.transform(test[["Breach_Source"]].astype(str))
        Xtr = np.concatenate([Xtr_num, Ptr], axis=1)
        Xva = np.concatenate([Xva_num, Pva], axis=1)
        Xte = np.concatenate([Xte_num, Pte], axis=1)
    else:
        Xtr, Xva, Xte = Xtr_num, Xva_num, Xte_num

    ytr = train["y"].astype(int).values
    yva = val["y"].astype(int).values
    yte = test["y"].astype(int).values

    meta = {
        "feat_cols": feat_cols,
        "setting": setting,
        "platform_onehot_dim": int(Xtr.shape[1] - len(feat_cols)) if setting == "cross_platform" else 0,
        "X_dim": int(Xtr.shape[1]),
    }
    return Xtr, ytr, Xva, yva, Xte, yte, platform_encoder, meta


# -------------------------
# Metrics (Accuracy-based threshold)
# -------------------------

def choose_threshold_by_accuracy(y_true: np.ndarray, y_prob: np.ndarray) -> float:
    """Find optimal threshold by ACCURACY on validation set."""
    best_thr, best_acc = 0.5, -1.0
    for thr in np.linspace(0.05, 0.95, 19):
        y_pred = (y_prob >= thr).astype(int)
        acc = accuracy_score(y_true, y_pred)
        if acc > best_acc:
            best_acc, best_thr = acc, float(thr)
    return best_thr

def compute_metrics(y_true: np.ndarray, y_prob: np.ndarray, thr: float) -> Dict[str, Any]:
    """Compute evaluation metrics (accuracy, F1, ROC-AUC, etc.)."""
    y_pred = (y_prob >= thr).astype(int)
    acc = float(accuracy_score(y_true, y_pred))
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)

    out = {
        "threshold": float(thr),
        "accuracy": acc,
        "precision": float(p),
        "recall": float(r),
        "f1": float(f1),
        "confusion_matrix": confusion_matrix(y_true, y_pred).tolist(),
    }

    try:
        out["auroc"] = float(roc_auc_score(y_true, y_prob))
    except Exception:
        out["auroc"] = None
    try:
        out["auprc"] = float(average_precision_score(y_true, y_prob))
    except Exception:
        out["auprc"] = None

    return out


# -------------------------
# Sequence Dataset
# -------------------------

class SeqDataset(Dataset):
    def __init__(self, X_seq: np.ndarray, y: np.ndarray):
        self.X = torch.tensor(X_seq, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self) -> int:
        return self.X.shape[0]

    def __getitem__(self, idx: int):
        return self.X[idx], self.y[idx]


def make_sequences_by_email(
    df: pd.DataFrame,
    feature_cols: List[str],
    seq_len: int,
    include_current: bool = True,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Build sequences per email for RNN models."""
    df = df.sort_values(["Email", "AttackTime"], kind="mergesort")
    F = len(feature_cols)

    X_list, y_list, idx_list = [], [], []

    for email, g in df.groupby("Email", sort=False):
        g = g.sort_values("AttackTime", kind="mergesort")
        feats = g[feature_cols].astype(float).values
        ys = g["y"].astype(int).values
        inds = g.index.values

        for i in range(len(g)):
            end = i + 1 if include_current else i
            start = max(0, end - seq_len)
            seq = feats[start:end]

            if seq.shape[0] < seq_len:
                pad = np.zeros((seq_len - seq.shape[0], F), dtype=np.float32)
                seq = np.vstack([pad, seq]).astype(np.float32)

            X_list.append(seq)
            y_list.append(ys[i])
            idx_list.append(inds[i])

    return (
        np.stack(X_list, axis=0).astype(np.float32),
        np.array(y_list, dtype=np.int64),
        np.array(idx_list, dtype=np.int64),
    )


# -------------------------
# PyTorch Models (Sequence)
# -------------------------

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim: int, hidden: int = 128, layers: int = 2, dropout: float = 0.2):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden, num_layers=layers, batch_first=True, dropout=dropout)
        self.head = nn.Sequential(
            nn.Linear(hidden, hidden // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden // 2, 1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.rnn(x)
        last = out[:, -1, :]
        return self.head(last).squeeze(-1)


class GRUClassifier(nn.Module):
    def __init__(self, input_dim: int, hidden: int = 128, layers: int = 2, dropout: float = 0.2):
        super().__init__()
        self.rnn = nn.GRU(input_dim, hidden, num_layers=layers, batch_first=True, dropout=dropout)
        self.head = nn.Sequential(
            nn.Linear(hidden, hidden // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden // 2, 1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.rnn(x)
        last = out[:, -1, :]
        return self.head(last).squeeze(-1)


class FTAGru(nn.Module):
    """
    - Feature attention per timestep
    - Temporal GRU encoder
    - Temporal attention pooling
    """
    def __init__(self, input_dim: int, hidden: int = 128, attn_dim: int = 64, dropout: float = 0.2):
        super().__init__()
        self.feature_gate = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.Sigmoid()
        )
        self.gru = nn.GRU(input_dim, hidden, num_layers=1, batch_first=True)
        self.temporal_attn = nn.Sequential(
            nn.Linear(hidden, attn_dim),
            nn.Tanh(),
            nn.Linear(attn_dim, 1)
        )
        self.head = nn.Sequential(
            nn.Linear(hidden, hidden // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden // 2, 1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        gates = self.feature_gate(x)
        xg = x * gates
        h, _ = self.gru(xg)
        a = self.temporal_attn(h).squeeze(-1)
        w = torch.softmax(a, dim=1).unsqueeze(-1)
        pooled = (h * w).sum(dim=1)
        return self.head(pooled).squeeze(-1)


class FTALstm(nn.Module):
    """
    - Feature attention per timestep
    - Temporal LSTM encoder
    - Temporal attention pooling
    """
    def __init__(self, input_dim: int, hidden: int = 128, attn_dim: int = 64, dropout: float = 0.2):
        super().__init__()
        self.feature_gate = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.Sigmoid()
        )
        self.lstm = nn.LSTM(input_dim, hidden, num_layers=1, batch_first=True)
        self.temporal_attn = nn.Sequential(
            nn.Linear(hidden, attn_dim),
            nn.Tanh(),
            nn.Linear(attn_dim, 1)
        )
        self.head = nn.Sequential(
            nn.Linear(hidden, hidden // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden // 2, 1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        gates = self.feature_gate(x)
        xg = x * gates
        h, _ = self.lstm(xg)
        a = self.temporal_attn(h).squeeze(-1)
        w = torch.softmax(a, dim=1).unsqueeze(-1)
        pooled = (h * w).sum(dim=1)
        return self.head(pooled).squeeze(-1)

# -------------------------
# IFCNN_TPP (CNN + GRU + attention pooling)
# -------------------------

class IFCNNTpp(nn.Module):
    """
    IFCNN_TPP-style binary classifier:
      - 1D conv over time (feature channels)
      - GRU encoder
      - Attention pooling
      - MLP head -> logits

    """
    def __init__(
        self,
        input_dim: int,
        hidden: int = 128,
        conv_channels: int = 128,
        kernel_size: int = 3,
        gru_layers: int = 1,
        attn_dim: int = 64,
        dropout: float = 0.2,
    ):
        super().__init__()
        padding = kernel_size // 2

        # Conv expects (B, C, T) where C=input_dim (treat features as channels)
        self.conv = nn.Sequential(
            nn.Conv1d(input_dim, conv_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(conv_channels, conv_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.Dropout(dropout),
        )

        self.proj = nn.Linear(conv_channels, hidden)

        self.gru = nn.GRU(
            input_size=hidden,
            hidden_size=hidden,
            num_layers=gru_layers,
            batch_first=True,
            dropout=dropout if gru_layers > 1 else 0.0,
        )

        self.temporal_attn = nn.Sequential(
            nn.Linear(hidden, attn_dim),
            nn.Tanh(),
            nn.Linear(attn_dim, 1),
        )

        self.head = nn.Sequential(
            nn.Linear(hidden, hidden // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden // 2, 1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, T, F)
        x = x.transpose(1, 2)                 # (B, F, T)
        z = self.conv(x)                      # (B, C, T)
        z = z.transpose(1, 2)                 # (B, T, C)
        z = self.proj(z)                      # (B, T, H)

        h, _ = self.gru(z)                    # (B, T, H)

        a = self.temporal_attn(h).squeeze(-1) # (B, T)
        w = torch.softmax(a, dim=1).unsqueeze(-1)
        pooled = (h * w).sum(dim=1)           # (B, H)

        logits = self.head(pooled).squeeze(-1)
        return logits


# -------------------------
# CEP3 (Causal dilated conv stack + attention pooling)
# -------------------------

class CEP3(nn.Module):
    """
    CEP3-style temporal classifier:
      - linear projection to hidden
      - 3-layer causal-ish dilated conv over time
      - attention pooling
      - MLP head -> logits
    """
    def __init__(
        self,
        input_dim: int,
        hidden: int = 128,
        kernel_size: int = 3,
        dropout: float = 0.2,
        attn_dim: int = 64,
    ):
        super().__init__()
        self.in_proj = nn.Linear(input_dim, hidden)

        # We'll do "same" padding then mask via slicing for a causal feel.
        # This is stable and avoids heavy TPP math.
        def conv_block(dilation: int):
            pad = (kernel_size - 1) * dilation
            return nn.Sequential(
                nn.Conv1d(hidden, hidden, kernel_size=kernel_size, dilation=dilation, padding=pad),
                nn.ReLU(),
                nn.Dropout(dropout),
            )

        self.conv1 = conv_block(dilation=1)
        self.conv2 = conv_block(dilation=2)
        self.conv3 = conv_block(dilation=4)

        self.temporal_attn = nn.Sequential(
            nn.Linear(hidden, attn_dim),
            nn.Tanh(),
            nn.Linear(attn_dim, 1),
        )

        self.head = nn.Sequential(
            nn.Linear(hidden, hidden // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden // 2, 1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, T, F)
        h = self.in_proj(x)                   # (B, T, H)
        h = h.transpose(1, 2)                 # (B, H, T)

        # Convs with padding; slice to keep length T (causal-ish)
        T = h.size(-1)
        h1 = self.conv1(h)[..., :T]
        h2 = self.conv2(h1)[..., :T]
        h3 = self.conv3(h2)[..., :T]

        out = h3.transpose(1, 2)              # (B, T, H)

        a = self.temporal_attn(out).squeeze(-1)  # (B, T)
        w = torch.softmax(a, dim=1).unsqueeze(-1)
        pooled = (out * w).sum(dim=1)            # (B, H)

        logits = self.head(pooled).squeeze(-1)
        return logits


# -------------------------
# Temporal GNN (timesteps-as-nodes) for sequences
# -------------------------

class TemporalGraphAttentionLayer(nn.Module):
    """
    Dense (batch) graph-attention over timesteps.
    Treat each timestep as a node; computes attention between all timestep pairs.
    """
    def __init__(self, in_features: int, out_features: int, dropout: float = 0.2, alpha: float = 0.2):
        super().__init__()
        self.dropout = dropout
        self.W = nn.Parameter(torch.empty(in_features, out_features))
        nn.init.xavier_uniform_(self.W, gain=1.414)

        self.a = nn.Parameter(torch.empty(2 * out_features, 1))
        nn.init.xavier_uniform_(self.a, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(alpha)

    def forward(self, x: torch.Tensor, adj: Optional[torch.Tensor] = None) -> torch.Tensor:
        """
        x: (B, T, Fin)
        adj: optional (T, T) or (B, T, T) binary mask where 0 means "no edge"
        returns: (B, T, Fout)
        """
        B, T, _ = x.shape
        h = torch.matmul(x, self.W)  # (B, T, Fout)

        # Build pairwise concat (B, T, T, 2*Fout)
        h1 = h.unsqueeze(2).expand(B, T, T, h.size(-1))
        h2 = h.unsqueeze(1).expand(B, T, T, h.size(-1))
        hcat = torch.cat([h1, h2], dim=-1)

        e = self.leakyrelu(torch.matmul(hcat, self.a).squeeze(-1))  # (B, T, T)

        if adj is not None:
            if adj.dim() == 2:
                adj = adj.unsqueeze(0)  # (1, T, T)
            e = e.masked_fill(adj == 0, -1e9)

        attn = torch.softmax(e, dim=-1)  # along neighbors
        attn = F.dropout(attn, self.dropout, training=self.training)

        out = torch.matmul(attn, h)  # (B, T, Fout)
        return out


class TemporalGNNClassifier(nn.Module):
    """
    Temporal GNN classifier:
      - GRU feature extraction over time
      - Multi-head temporal graph attention (timesteps as nodes)
      - Temporal self-attention
      - Mean pooling -> MLP head

    Works with your SeqDataset and train_torch_binary().
    """
    def __init__(
        self,
        input_dim: int,
        hidden: int = 128,
        gru_layers: int = 2,
        gat_heads: int = 4,
        dropout: float = 0.2,
    ):
        super().__init__()
        self.hidden = hidden
        self.gat_heads = gat_heads

        self.gru = nn.GRU(
            input_dim,
            hidden,
            num_layers=gru_layers,
            batch_first=True,
            dropout=dropout if gru_layers > 1 else 0.0,
        )

        # Multi-head temporal graph attention
        # Each head outputs hidden/gat_heads dims; concatenation returns hidden
        head_dim = max(1, hidden // gat_heads)
        self.gat_layers = nn.ModuleList([
            TemporalGraphAttentionLayer(hidden, head_dim, dropout=dropout)
            for _ in range(gat_heads)
        ])

        self.layer_norm = nn.LayerNorm(head_dim * gat_heads)

        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=head_dim * gat_heads,
            num_heads=4,
            dropout=dropout,
            batch_first=True,
        )

        self.dropout = nn.Dropout(dropout)
        self.head = nn.Sequential(
            nn.Linear(head_dim * gat_heads, (head_dim * gat_heads) // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear((head_dim * gat_heads) // 2, 1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, T, Fin)
        h, _ = self.gru(x)  # (B, T, hidden)

        # Dense temporal GAT over timesteps
        gat_outs = [gat(h) for gat in self.gat_layers]          # list of (B, T, head_dim)
        h_gat = torch.cat(gat_outs, dim=-1)                     # (B, T, head_dim*heads)

        # Residual-ish normalization (dims match by construction)
        h_gat = self.layer_norm(h_gat)

        # Temporal self-attention
        attn_out, _ = self.temporal_attention(h_gat, h_gat, h_gat)

        # Pool and classify
        pooled = attn_out.mean(dim=1)                           # (B, D)
        pooled = self.dropout(pooled)
        logits = self.head(pooled).squeeze(-1)                  # (B,)
        return logits

# -------------------------
# Graph Components (Bipartite Email-Platform)
# -------------------------

def build_bipartite_graph(train_df: pd.DataFrame) -> Tuple[Dict[str, int], Dict[str, int], torch.Tensor]:
    """Build bipartite email-platform graph."""
    emails = sorted(train_df["Email"].astype(str).unique().tolist())
    plats = sorted(train_df["Breach_Source"].astype(str).unique().tolist())
    email2n = {e: i for i, e in enumerate(emails)}
    plat2n = {p: i for i, p in enumerate(plats)}

    E, P = len(emails), len(plats)
    edges = set()

    for _, r in train_df[["Email", "Breach_Source"]].drop_duplicates().iterrows():
        u = email2n[str(r["Email"])]
        v = E + plat2n[str(r["Breach_Source"])]
        edges.add((u, v))
        edges.add((v, u))

    if not edges:
        raise ValueError("No edges in bipartite graph")

    idx = torch.tensor(list(edges), dtype=torch.long).t().contiguous()
    N = E + P
    val = torch.ones(idx.shape[1], dtype=torch.float32)
    A = torch.sparse_coo_tensor(idx, val, (N, N))

    return email2n, plat2n, A.coalesce()


def normalize_sparse_adj(A: torch.Tensor) -> torch.Tensor:
    """Symmetric normalization: D^{-1/2} A D^{-1/2}"""
    A = A.coalesce()
    idx = A.indices()
    val = A.values()
    N = A.shape[0]

    deg = torch.zeros(N, dtype=torch.float32, device=val.device)
    deg.index_add_(0, idx[0], val)
    deg_inv_sqrt = torch.pow(deg.clamp(min=1.0), -0.5)
    norm_val = deg_inv_sqrt[idx[0]] * val * deg_inv_sqrt[idx[1]]

    return torch.sparse_coo_tensor(idx, norm_val, (N, N)).coalesce()


# -------------------------
# GCN Components
# -------------------------

class GCNLayer(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        super().__init__()
        self.lin = nn.Linear(in_dim, out_dim)

    def forward(self, A_norm: torch.Tensor, X: torch.Tensor) -> torch.Tensor:
        # Sparse matmul doesn't support fp16 on many CUDA builds.
        # Force this operation to fp32 even if AMP/autocast is enabled.
        if X.is_cuda:
            with torch.cuda.amp.autocast(enabled=False):
                AX = torch.sparse.mm(A_norm.float(), X.float())
        else:
            AX = torch.sparse.mm(A_norm, X)
        return self.lin(AX)


class BipartiteGCNClassifier(nn.Module):
    def __init__(self, node_in_dim: int, gcn_hidden: int, gcn_out: int, event_in_dim: int, dropout: float = 0.2):
        super().__init__()
        self.gcn1 = GCNLayer(node_in_dim, gcn_hidden)
        self.gcn2 = GCNLayer(gcn_hidden, gcn_out)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.mlp = nn.Sequential(
            nn.Linear(event_in_dim + 2 * gcn_out, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 1),
        )

    def forward(
        self,
        A_norm: torch.Tensor,
        X_nodes: torch.Tensor,
        email_nodes: torch.Tensor,
        plat_nodes: torch.Tensor,
        X_event: torch.Tensor,
    ) -> torch.Tensor:
        h = self.act(self.gcn1(A_norm, X_nodes))
        h = self.dropout(h)
        h = self.gcn2(A_norm, h)

        he = h[email_nodes]
        hp = h[plat_nodes]
        z = torch.cat([X_event, he, hp], dim=1)
        return self.mlp(z).squeeze(-1)


# -------------------------
# GAT Components (COO edges; AMP-safe softmax)
# -------------------------

class GATLayer(nn.Module):
    """
    Sparse GAT layer using COO edge list.
    Computes attention only over edges in edge_index.
    """
    def __init__(self, in_dim: int, out_dim: int, dropout: float = 0.2, negative_slope: float = 0.2):
        super().__init__()
        self.W = nn.Linear(in_dim, out_dim, bias=False)
        self.attn = nn.Linear(2 * out_dim, 1, bias=False)
        self.leakyrelu = nn.LeakyReLU(negative_slope)
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def _segment_softmax(scores: torch.Tensor, dst: torch.Tensor, num_nodes: int) -> torch.Tensor:
        """
        Softmax over incoming edges per destination node.
        Runs in FP32 for numerical stability under AMP, then casts back.
        """
        orig_dtype = scores.dtype
        scores_f = scores.float()  # fp32

        max_per_dst = torch.full((num_nodes,), -1e9, device=scores.device, dtype=torch.float32)
        max_per_dst.scatter_reduce_(0, dst, scores_f, reduce="amax", include_self=True)

        scores_f = (scores_f - max_per_dst[dst]).clamp(min=-20.0, max=20.0)
        scores_exp = torch.exp(scores_f)

        denom = torch.zeros((num_nodes,), device=scores.device, dtype=torch.float32)
        denom.scatter_add_(0, dst, scores_exp)

        out = scores_exp / denom[dst].clamp_min(1e-12)
        return out.to(orig_dtype)

    def forward(self, X: torch.Tensor, edge_index: torch.Tensor, num_nodes: int) -> torch.Tensor:
        H = self.W(X)  # [N, Fout]

        src, dst = edge_index[0], edge_index[1]  # [E], [E]
        h_src = H[src]
        h_dst = H[dst]

        e = self.attn(torch.cat([h_src, h_dst], dim=1)).squeeze(-1)
        e = self.leakyrelu(e)

        alpha = self._segment_softmax(e, dst, num_nodes)
        alpha = self.dropout(alpha)

        out = torch.zeros((num_nodes, H.shape[1]), device=H.device, dtype=H.dtype)
        out.index_add_(0, dst, h_src * alpha.unsqueeze(-1))
        return out


class MultiHeadGATLayer(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, heads: int = 4, dropout: float = 0.2):
        super().__init__()
        self.heads = nn.ModuleList([GATLayer(in_dim, out_dim, dropout=dropout) for _ in range(heads)])

    def forward(self, X: torch.Tensor, edge_index: torch.Tensor, num_nodes: int) -> torch.Tensor:
        outs = [h(X, edge_index, num_nodes) for h in self.heads]
        return torch.cat(outs, dim=1)


class BipartiteGATClassifier(nn.Module):
    """Bipartite GAT classifier over the email-platform graph."""
    def __init__(
        self,
        node_in_dim: int,
        gat_hidden: int,
        gat_out: int,
        heads1: int,
        heads2: int,
        event_in_dim: int,
        dropout: float = 0.2,
    ):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.act = nn.ELU()

        self.gat1 = MultiHeadGATLayer(node_in_dim, gat_hidden, heads=heads1, dropout=dropout)
        self.gat2 = MultiHeadGATLayer(gat_hidden * heads1, gat_out, heads=heads2, dropout=dropout)

        gnn_dim = gat_out * heads2
        self.mlp = nn.Sequential(
            nn.Linear(event_in_dim + 2 * gnn_dim, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 1),
        )

    def forward(
        self,
        edge_index: torch.Tensor,
        X_nodes: torch.Tensor,
        email_nodes: torch.Tensor,
        plat_nodes: torch.Tensor,
        X_event: torch.Tensor,
    ) -> torch.Tensor:
        N = X_nodes.shape[0]
        h = self.gat1(X_nodes, edge_index, N)
        h = self.act(h)
        h = self.dropout(h)
        h = self.gat2(h, edge_index, N)
        h = self.act(h)

        he = h[email_nodes]
        hp = h[plat_nodes]
        z = torch.cat([X_event, he, hp], dim=1)
        return self.mlp(z).squeeze(-1)


class GraphDataset(Dataset):
    def __init__(self, X_event: np.ndarray, y: np.ndarray, email_nodes: np.ndarray, plat_nodes: np.ndarray):
        self.Xe = torch.tensor(X_event, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.en = torch.tensor(email_nodes, dtype=torch.long)
        self.pn = torch.tensor(plat_nodes, dtype=torch.long)

    def __len__(self) -> int:
        return self.Xe.shape[0]

    def __getitem__(self, idx: int):
        return self.Xe[idx], self.y[idx], self.en[idx], self.pn[idx]


# -------------------------
# Training Loops with AMP (Accuracy Early Stop)
# -------------------------

def train_torch_binary(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    lr: float = 1e-3,
    weight_decay: float = 1e-5,
    epochs: int = 30,
    patience: int = 8,
    class_weight_pos: Optional[float] = None,
    use_amp: bool = True,
) -> Tuple[nn.Module, Dict[str, Any]]:
    """Train sequence model with AMP support. Uses ACCURACY for early stopping."""
    model = model.to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    use_amp = use_amp and device.type == "cuda"
    scaler = GradScaler() if use_amp else None

    if class_weight_pos is None:
        bce = nn.BCEWithLogitsLoss()
    else:
        pos_weight = torch.tensor([class_weight_pos], dtype=torch.float32, device=device)
        bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    best_state = None
    best_val_acc = -1.0
    bad_epochs = 0
    history = {"val_acc": [], "val_f1": [], "val_loss": [], "train_loss": []}

    for ep in range(1, epochs + 1):
        model.train()
        train_losses = []

        for xb, yb in train_loader:
            xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
            opt.zero_grad()

            if use_amp:
                with autocast():
                    logits = model(xb)
                    loss = bce(logits, yb)
                scaler.scale(loss).backward()
                scaler.unscale_(opt)
                nn.utils.clip_grad_norm_(model.parameters(), 5.0)
                scaler.step(opt)
                scaler.update()
            else:
                logits = model(xb)
                loss = bce(logits, yb)
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), 5.0)
                opt.step()

            train_losses.append(float(loss.item()))

        model.eval()
        all_prob, all_y, val_losses = [], [], []

        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)

                if use_amp:
                    with autocast():
                        logits = model(xb)
                        loss = bce(logits, yb)
                else:
                    logits = model(xb)
                    loss = bce(logits, yb)

                val_losses.append(float(loss.item()))
                prob = torch.sigmoid(logits).cpu().numpy()
                all_prob.append(prob)
                all_y.append(yb.cpu().numpy())

        y_prob = np.concatenate(all_prob)
        y_true = np.concatenate(all_y).astype(int)

        thr = choose_threshold_by_accuracy(y_true, y_prob)
        met = compute_metrics(y_true, y_prob, thr)
        val_acc = met["accuracy"]
        val_f1 = met["f1"]

        history["val_acc"].append(val_acc)
        history["val_f1"].append(val_f1)
        history["val_loss"].append(float(np.mean(val_losses)))
        history["train_loss"].append(float(np.mean(train_losses)))

        print(
            f"  Epoch {ep}: "
            f"train_loss={np.mean(train_losses):.4f}, "
            f"val_loss={np.mean(val_losses):.4f}, "
            f"val_acc={val_acc:.4f}, val_f1={val_f1:.4f}"
        )

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                print(f"  Early stopping at epoch {ep}")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, {"best_val_acc": best_val_acc, "history": history}


def predict_torch_seq(model: nn.Module, loader: DataLoader, device: torch.device, use_amp: bool = True) -> np.ndarray:
    """Predict with sequence model."""
    model.eval()
    use_amp = use_amp and device.type == "cuda"
    probs = []

    with torch.no_grad():
        for xb, _ in loader:
            xb = xb.to(device, non_blocking=True)
            if use_amp:
                with autocast():
                    logits = model(xb)
            else:
                logits = model(xb)
            prob = torch.sigmoid(logits).cpu().numpy()
            probs.append(prob)

    return np.concatenate(probs).astype(float)


def train_graph_binary(
    model: nn.Module,
    graph_obj: torch.Tensor,        # A_norm (sparse) for GCN or edge_index (long) for GAT
    X_nodes: torch.Tensor,
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    lr: float = 1e-3,
    weight_decay: float = 1e-5,
    epochs: int = 30,
    patience: int = 8,
    class_weight_pos: Optional[float] = None,
    use_amp: bool = True,
) -> Tuple[nn.Module, Dict[str, Any]]:
    """Train GCN/GAT classifier with AMP support. Uses ACCURACY for early stopping."""
    model = model.to(device)
    graph_obj = graph_obj.to(device)
    X_nodes = X_nodes.to(device)

    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    use_amp = use_amp and device.type == "cuda"
    scaler = GradScaler() if use_amp else None

    if class_weight_pos is None:
        bce = nn.BCEWithLogitsLoss()
    else:
        pos_weight = torch.tensor([class_weight_pos], dtype=torch.float32, device=device)
        bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    best_state = None
    best_val_acc = -1.0
    bad_epochs = 0
    history = {"val_acc": [], "val_f1": []}

    for ep in range(1, epochs + 1):
        model.train()

        for Xe, yb, en, pn in train_loader:
            Xe, yb = Xe.to(device, non_blocking=True), yb.to(device, non_blocking=True)
            en, pn = en.to(device, non_blocking=True), pn.to(device, non_blocking=True)

            opt.zero_grad()

            if use_amp:
                with autocast():
                    logits = model(graph_obj, X_nodes, en, pn, Xe)
                    loss = bce(logits, yb)
                scaler.scale(loss).backward()
                scaler.unscale_(opt)
                nn.utils.clip_grad_norm_(model.parameters(), 5.0)
                scaler.step(opt)
                scaler.update()
            else:
                logits = model(graph_obj, X_nodes, en, pn, Xe)
                loss = bce(logits, yb)
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), 5.0)
                opt.step()

        model.eval()
        all_prob, all_y = [], []

        with torch.no_grad():
            for Xe, yb, en, pn in val_loader:
                Xe, yb = Xe.to(device, non_blocking=True), yb.to(device, non_blocking=True)
                en, pn = en.to(device, non_blocking=True), pn.to(device, non_blocking=True)

                if use_amp:
                    with autocast():
                        logits = model(graph_obj, X_nodes, en, pn, Xe)
                else:
                    logits = model(graph_obj, X_nodes, en, pn, Xe)

                prob = torch.sigmoid(logits).cpu().numpy()
                all_prob.append(prob)
                all_y.append(yb.cpu().numpy())

        y_prob = np.concatenate(all_prob)
        y_true = np.concatenate(all_y).astype(int)

        thr = choose_threshold_by_accuracy(y_true, y_prob)
        met = compute_metrics(y_true, y_prob, thr)
        val_acc = met["accuracy"]
        val_f1 = met["f1"]

        history["val_acc"].append(val_acc)
        history["val_f1"].append(val_f1)

        print(f"  Epoch {ep}: val_acc={val_acc:.4f}, val_f1={val_f1:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                print(f"  Early stopping at epoch {ep}")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, {"best_val_acc": best_val_acc, "history": history}

 =============================================================================
# SUMMARY TABLE PRINTING (requested)
# =============================================================================
def predict_graph(
    model: nn.Module,
    graph_obj: torch.Tensor,
    X_nodes: torch.Tensor,
    loader: DataLoader,
    device: torch.device,
    use_amp: bool = True
) -> np.ndarray:
    """Predict with GCN/GAT model."""
    model.eval()
    graph_obj = graph_obj.to(device)
    X_nodes = X_nodes.to(device)

    use_amp = use_amp and device.type == "cuda"
    probs = []

    with torch.no_grad():
        for Xe, _, en, pn in loader:
            Xe = Xe.to(device, non_blocking=True)
            en, pn = en.to(device, non_blocking=True), pn.to(device, non_blocking=True)

            if use_amp:
                with autocast():
                    logits = model(graph_obj, X_nodes, en, pn, Xe)
            else:
                logits = model(graph_obj, X_nodes, en, pn, Xe)

            prob = torch.sigmoid(logits).cpu().numpy()
            probs.append(prob)

    return np.concatenate(probs).astype(float)


# -------------------------
# Main Training Orchestration
# -------------------------

def train_and_evaluate(
    train: pd.DataFrame,
    val: pd.DataFrame,
    test: pd.DataFrame,
    out_dir: str,
    setting: str,
    model_name: str,
    seq_len: int = 10,
    batch: int = 256,
    epochs: int = 30,
    lr: float = 1e-3,
    weight_decay: float = 1e-5,
    patience: int = 8,
    seed: int = 42,
    device: Optional[torch.device] = None,
    use_amp: bool = True,
) -> Dict[str, Any]:
    ensure_dir(out_dir)
    seed_everything(seed)

    if device is None:
        device = get_device()

    results = {
        "setting": setting,
        "model": model_name,
        "seq_len": seq_len,
        "seed": seed,
        "device": str(device),
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    }

    if setting == "per_platform":
        all_platforms = sorted(train["Breach_Source"].astype(str).unique().tolist())
        per_plat_metrics = {}
        per_plat_pred_files = {}

        for plat in all_platforms:
            trp = train[train["Breach_Source"] == plat].copy()
            vap = val[val["Breach_Source"] == plat].copy()
            tep = test[test["Breach_Source"] == plat].copy()

            if len(trp) < 50 or len(vap) < 20 or len(tep) < 20:
                print(f"Skipping platform {plat}: insufficient data")
                continue

            print(f"\nTraining for platform: {plat}")
            plat_out = os.path.join(out_dir, f"per_platform_{plat}")
            ensure_dir(plat_out)

            met, pred_path, thr = _train_eval_single(
                trp, vap, tep, plat_out, setting="per_platform", model_name=model_name,
                seq_len=seq_len, batch=batch, epochs=epochs, lr=lr,
                weight_decay=weight_decay, patience=patience, seed=seed,
                device=device, use_amp=use_amp
            )
            per_plat_metrics[plat] = met
            per_plat_pred_files[plat] = {"predictions_csv": pred_path, "threshold": thr}

            # Optional cleanup for long loops
            import gc
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        results["per_platform_metrics"] = per_plat_metrics
        results["per_platform_predictions"] = per_plat_pred_files
        save_json(results, os.path.join(out_dir, f"results_{setting}_{model_name}.json"))
        return results

    met, pred_path, thr = _train_eval_single(
        train, val, test, out_dir, setting=setting, model_name=model_name,
        seq_len=seq_len, batch=batch, epochs=epochs, lr=lr,
        weight_decay=weight_decay, patience=patience, seed=seed,
        device=device, use_amp=use_amp
    )
    results["metrics"] = met
    results["predictions_csv"] = pred_path
    results["threshold"] = thr
    save_json(results, os.path.join(out_dir, f"results_{setting}_{model_name}.json"))

    return results


def _train_eval_single(
    train: pd.DataFrame,
    val: pd.DataFrame,
    test: pd.DataFrame,
    out_dir: str,
    setting: str,
    model_name: str,
    seq_len: int,
    batch: int,
    epochs: int,
    lr: float,
    weight_decay: float,
    patience: int,
    seed: int,
    device: torch.device,
    use_amp: bool = True,
) -> Tuple[Dict[str, Any], str, float]:
    ensure_dir(out_dir)

    Xtr, ytr, Xva, yva, Xte, yte, platform_encoder, meta = build_design_matrices(
        train, val, test, setting=setting, platform_encoder=None
    )

    pos = max(int((ytr == 1).sum()), 1)
    neg = max(int((ytr == 0).sum()), 1)
    class_weight_pos = float(neg / pos)
    print(f"  Class balance: {pos} positive, {neg} negative (weight={class_weight_pos:.2f})")

    # -------------------------
    # Sequence models
    # -------------------------
    if model_name in {"lstm", "gru", "fta_gru", "fta_lstm", "gnn", "ifcnn_tpp", "cep3"}:
        print(f"Training {model_name.upper()}...")

        use_cross = (setting == "cross_platform")
        feature_cols = list(BASE_FEATURES) + (list(CROSS_FEATURES) if use_cross else [])

        if setting == "cross_platform":
            platform_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
            platform_encoder.fit(train[["Breach_Source"]].astype(str))

            def add_plat_onehot(d: pd.DataFrame):
                oh = platform_encoder.transform(d[["Breach_Source"]].astype(str))
                oh_cols = [f"plat_oh_{i}" for i in range(oh.shape[1])]
                dd = d.copy()
                for j, c in enumerate(oh_cols):
                    dd[c] = oh[:, j]
                return dd, oh_cols

            train2, oh_cols = add_plat_onehot(train)
            val2, _ = add_plat_onehot(val)
            test2, _ = add_plat_onehot(test)
            feature_cols = feature_cols + oh_cols
        else:
            train2, val2, test2 = train, val, test

        Xtr_seq, ytr_seq, idx_tr = make_sequences_by_email(train2, feature_cols, seq_len=seq_len)
        Xva_seq, yva_seq, idx_va = make_sequences_by_email(val2, feature_cols, seq_len=seq_len)
        Xte_seq, yte_seq, idx_te = make_sequences_by_email(test2, feature_cols, seq_len=seq_len)

        tr_loader = make_loader(SeqDataset(Xtr_seq, ytr_seq), batch, True,  device, setting)
        va_loader = make_loader(SeqDataset(Xva_seq, yva_seq), batch, False, device, setting)
        te_loader = make_loader(SeqDataset(Xte_seq, yte_seq), batch, False, device, setting)

        input_dim = Xtr_seq.shape[-1]

        if model_name == "lstm":
            net = LSTMClassifier(input_dim=input_dim, hidden=128, layers=2, dropout=0.2)
        elif model_name == "gru":
            net = GRUClassifier(input_dim=input_dim, hidden=128, layers=2, dropout=0.2)
        elif model_name == "fta_gru":
            net = FTAGru(input_dim=input_dim, hidden=128, attn_dim=64, dropout=0.2)
        elif model_name == "fta_lstm":
            net = FTALstm(input_dim=input_dim, hidden=128, attn_dim=64, dropout=0.2)
        elif model_name == "gnn":
            net = TemporalGNNClassifier(input_dim=input_dim, hidden=128, gru_layers=2, gat_heads=4, dropout=0.2)
        elif model_name == "ifcnn_tpp":
            net = IFCNNTpp(input_dim=input_dim, hidden=128, conv_channels=128, kernel_size=3, gru_layers=1, dropout=0.2)
        elif model_name == "cep3":
            net = CEP3(input_dim=input_dim, hidden=128, kernel_size=3, dropout=0.2)
        else:
            raise ValueError(f"Unhandled model_name: {model_name}")


        net, train_info = train_torch_binary(
            net, tr_loader, va_loader, device=device,
            lr=lr, weight_decay=weight_decay, epochs=epochs, patience=patience,
            class_weight_pos=class_weight_pos, use_amp=use_amp
        )

        va_prob = predict_torch_seq(net, va_loader, device=device, use_amp=use_amp)
        best_thr = choose_threshold_by_accuracy(yva_seq, va_prob)
        te_prob = predict_torch_seq(net, te_loader, device=device, use_amp=use_amp)

        metrics = compute_metrics(yte_seq, te_prob, best_thr)
        metrics["train_info"] = train_info

        torch.save(net.state_dict(), os.path.join(out_dir, f"model_{model_name}.pt"))
        save_json({"meta": meta, "feature_cols": feature_cols}, os.path.join(out_dir, "meta.json"))

        test_pred_df = test2.loc[idx_te, ID_COLS + ["y"]].copy()
        test_pred_df["pred_prob"] = te_prob
        pred_path = os.path.join(out_dir, "predictions_test.csv")
        test_pred_df.to_csv(pred_path, index=False)

        return metrics, pred_path, best_thr

    # -------------------------
    # Graph models: GCN + GAT
    # -------------------------
    if model_name in {"gcn", "gat"}:
        if setting != "cross_platform":
            raise ValueError(f"{model_name.upper()} requires setting='cross_platform'")

        print(f"Training {model_name.upper()}...")

        platform_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
        platform_encoder.fit(train[["Breach_Source"]].astype(str))

        def event_X(d: pd.DataFrame) -> np.ndarray:
            Xn = d[list(BASE_FEATURES) + list(CROSS_FEATURES)].astype(float).values
            oh = platform_encoder.transform(d[["Breach_Source"]].astype(str))
            return np.concatenate([Xn, oh], axis=1)

        Xtr_e, Xva_e, Xte_e = event_X(train), event_X(val), event_X(test)
        ytr_e = train["y"].astype(int).values
        yva_e = val["y"].astype(int).values
        yte_e = test["y"].astype(int).values

        email2n, plat2n, A = build_bipartite_graph(train)
        A_norm = normalize_sparse_adj(A)
        edge_index = A.indices()  # COO edges for GAT

        E, P = len(email2n), len(plat2n)
        N = E + P

        node_in_dim = Xtr_e.shape[1]
        X_nodes = np.zeros((N, node_in_dim), dtype=np.float32)
        counts = np.zeros(N, dtype=np.float32)

        # Aggregate event features into node features (mean per node)
        for i, r in train.iterrows():
            e, p = str(r["Email"]), str(r["Breach_Source"])
            pos_i = train.index.get_loc(i)

            if e in email2n:
                ni = email2n[e]
                X_nodes[ni] += Xtr_e[pos_i]
                counts[ni] += 1.0
            if p in plat2n:
                nj = E + plat2n[p]
                X_nodes[nj] += Xtr_e[pos_i]
                counts[nj] += 1.0

        counts = np.clip(counts, 1.0, None)
        X_nodes = (X_nodes / counts[:, None]).astype(np.float32)

        def map_nodes(d: pd.DataFrame):
            en = np.array([email2n.get(str(e), 0) for e in d["Email"].astype(str).values], dtype=np.int64)
            pn = np.array([E + plat2n.get(str(p), 0) for p in d["Breach_Source"].astype(str).values], dtype=np.int64)
            return en, pn

        en_tr, pn_tr = map_nodes(train)
        en_va, pn_va = map_nodes(val)
        en_te, pn_te = map_nodes(test)

        tr_loader = make_loader(GraphDataset(Xtr_e, ytr_e, en_tr, pn_tr), batch, True,  device, setting)
        va_loader = make_loader(GraphDataset(Xva_e, yva_e, en_va, pn_va), batch, False, device, setting)
        te_loader = make_loader(GraphDataset(Xte_e, yte_e, en_te, pn_te), batch, False, device, setting)

        if model_name == "gcn":
            net = BipartiteGCNClassifier(
                node_in_dim=node_in_dim,
                gcn_hidden=128,
                gcn_out=64,
                event_in_dim=Xtr_e.shape[1],
                dropout=0.2
            )
            graph_obj = A_norm
        else:
            net = BipartiteGATClassifier(
                node_in_dim=node_in_dim,
                gat_hidden=32,
                gat_out=32,
                heads1=4,
                heads2=4,
                event_in_dim=Xtr_e.shape[1],
                dropout=0.2
            )
            graph_obj = edge_index

        net, train_info = train_graph_binary(
            net,
            graph_obj=graph_obj,
            X_nodes=torch.tensor(X_nodes, dtype=torch.float32),
            train_loader=tr_loader,
            val_loader=va_loader,
            device=device,
            lr=lr,
            weight_decay=weight_decay,
            epochs=epochs,
            patience=patience,
            class_weight_pos=class_weight_pos,
            use_amp=use_amp
        )

        va_prob = predict_graph(net, graph_obj, torch.tensor(X_nodes, dtype=torch.float32),
                                va_loader, device=device, use_amp=use_amp)
        best_thr = choose_threshold_by_accuracy(yva_e, va_prob)

        te_prob = predict_graph(net, graph_obj, torch.tensor(X_nodes, dtype=torch.float32),
                                te_loader, device=device, use_amp=use_amp)

        metrics = compute_metrics(yte_e, te_prob, best_thr)
        metrics["train_info"] = train_info

        torch.save(net.state_dict(), os.path.join(out_dir, f"model_{model_name}.pt"))
        save_json({"meta": meta, "node_in_dim": node_in_dim, "email_count": E, "platform_count": P},
                  os.path.join(out_dir, "meta.json"))

        test_pred_df = test[ID_COLS + ["y"]].copy()
        test_pred_df["pred_prob"] = te_prob
        pred_path = os.path.join(out_dir, "predictions_test.csv")
        test_pred_df.to_csv(pred_path, index=False)

        return metrics, pred_path, best_thr

    raise ValueError(f"Unknown model: {model_name}")


# -------------------------
# Main
# -------------------------

def main():
    # =========================
    # USER CONFIGURATION
    # =========================
    DATA_DIR = "/home/Passwords/Analysis"
    OUT_DIR = "/home/Passwords/Analysis/Detection_Output"

    SETTINGS = ["cross_platform", "per_platform"]
    MODELS = ["fta_lstm", "fta_gru", "lstm", "gru", "gnn", "ifcnn_tpp", "cep3", "gcn", "gat"]

    SEQ_LEN = 10
    BATCH_SIZE = 256
    EPOCHS = 30
    LR = 1e-3
    WEIGHT_DECAY = 1e-5
    PATIENCE = 8
    SEED = 42

    DEVICE = None          # "cuda" | "cpu" | "mps" | None(auto)
    USE_AMP = True         # set False to disable
    FAIL_FAST = False      # True: stop on first failure; False: continue

    # =========================
    # DEVICE SETUP
    # =========================
    if DEVICE:
        device = torch.device(DEVICE)
    else:
        device = get_device()

    print(f"Using device: {device}")
    os.makedirs(OUT_DIR, exist_ok=True)

    # =========================
    # LOAD DATA ONCE
    # =========================
    print(f"Loading cached data from: {DATA_DIR}")
    train, val, test = load_cached(DATA_DIR)

    # =========================
    # RUN GRID: settings x models
    # =========================
    all_runs = []
    total = len(SETTINGS) * len(MODELS)
    run_idx = 0

    for setting in SETTINGS:
        for model_name in MODELS:
            if model_name in {"gcn", "gat"} and setting != "cross_platform":
                print(f"Skipping invalid combo: {setting} + {model_name}")
                continue

            run_idx += 1
            run_tag = f"{setting}__{model_name}"
            run_out_dir = os.path.join(OUT_DIR, run_tag)
            os.makedirs(run_out_dir, exist_ok=True)

            print("\n" + "=" * 80)
            print(f"[{run_idx}/{total}] Training model={model_name} setting={setting}")
            print(f"Run output dir: {run_out_dir}")
            print("=" * 80)

            started = time.time()
            run_record = {
                "setting": setting,
                "model": model_name,
                "out_dir": run_out_dir,
                "status": "started",
                "started_at": started,
            }

            try:
                results = train_and_evaluate(
                    train=train,
                    val=val,
                    test=test,
                    out_dir=run_out_dir,
                    setting=setting,
                    model_name=model_name,
                    seq_len=SEQ_LEN,
                    batch=BATCH_SIZE,
                    epochs=EPOCHS,
                    lr=LR,
                    weight_decay=WEIGHT_DECAY,
                    patience=PATIENCE,
                    seed=SEED,
                    device=device,
                    use_amp=USE_AMP,
                )

                elapsed = time.time() - started
                run_record["status"] = "ok"
                run_record["elapsed_sec"] = elapsed

                if isinstance(results, dict) and "metrics" in results and isinstance(results["metrics"], dict):
                    run_record["metrics"] = results["metrics"]
                    acc = results["metrics"].get("accuracy", None)
                    f1 = results["metrics"].get("f1", None)
                    auroc = results["metrics"].get("auroc", None)
                    auroc_str = f"{auroc:.4f}" if auroc is not None else "N/A"
                    print(f"Done: {run_tag} | elapsed={elapsed:.1f}s | ACC={acc:.4f} | F1={f1:.4f} | AUROC={auroc_str}")
                else:
                    run_record["metrics"] = None
                    print(f"Done: {run_tag} | elapsed={elapsed:.1f}s | (no metrics returned)")

                with open(os.path.join(run_out_dir, "run_summary.json"), "w") as f:
                    json.dump(run_record, f, indent=2)

            except Exception as e:
                elapsed = time.time() - started
                run_record["status"] = "error"
                run_record["elapsed_sec"] = elapsed
                run_record["error"] = str(e)
                run_record["traceback"] = traceback.format_exc()

                print(f"ERROR in {run_tag} after {elapsed:.1f}s: {e}")

                with open(os.path.join(run_out_dir, "run_summary.json"), "w") as f:
                    json.dump(run_record, f, indent=2)

                if FAIL_FAST:
                    raise

            # Per-run cleanup (helps long grids)
            import gc
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            all_runs.append(run_record)

    # =========================
    # SAVE MASTER SUMMARY
    # =========================
    master_path = os.path.join(OUT_DIR, "all_runs_summary.json")
    with open(master_path, "w") as f:
        json.dump(all_runs, f, indent=2)

    print("\n" + "=" * 80)
    print("ALL RUNS COMPLETE")
    print(f"Master summary: {master_path}")
    print("=" * 80)

    scored = []
    for r in all_runs:
        if r.get("status") == "ok" and isinstance(r.get("metrics"), dict):
            acc = r["metrics"].get("accuracy", None)
            f1 = r["metrics"].get("f1", None)
            auroc = r["metrics"].get("auroc", None)
            if acc is not None:
                scored.append((float(acc), f1, auroc, r["setting"], r["model"], r["out_dir"]))

    if scored:
        scored.sort(reverse=True)
        print("\nTop runs by Accuracy:")
        for i, (acc, f1, auroc, setting, model, out_dir) in enumerate(scored[:10], 1):
            f1_str = f"{f1:.4f}" if f1 is not None else "N/A"
            auroc_str = f"{auroc:.4f}" if auroc is not None else "N/A"
            print(f"{i:>2}. ACC={acc:.4f} F1={f1_str} AUROC={auroc_str} | {setting:>13} | {model:>8}")
    else:
        print("\nNo accuracy scores found in returned metrics.")


if __name__ == "__main__":
    main()

# Detection and Delay for Breach Dection Models

In [ ]:
import os
import json
import traceback
from typing import Dict, Any, List, Optional, Tuple
import pandas as pd
import numpy as np


# =============================================================================
# USER CONFIG (SET THIS)
# =============================================================================
OUT_DIR = "Detection_Output"
MODELS = ["fta_lstm", "fta_gru", "lstm", "gru", "gnn", "ifcnn_tpp", "cep3", "gcn", "gat"]
SETTINGS = ["per_platform", "cross_platform"]

STAGE3_DIR = os.path.join(OUT_DIR, "stage3")
DELAY_DIR = os.path.join(STAGE3_DIR, "delay")
BENEFIT_DIR = os.path.join(STAGE3_DIR, "benefit")
REPORT_PATH = os.path.join(STAGE3_DIR, "summary_report.json")
SUMMARY_TABLE_CSV = os.path.join(STAGE3_DIR, "summary_table.csv")

os.makedirs(DELAY_DIR, exist_ok=True)
os.makedirs(BENEFIT_DIR, exist_ok=True)


# =============================================================================
# SAFE HELPERS
# =============================================================================
def _exists_file(path: str) -> bool:
    if not path or not os.path.exists(path):
        print(f"[WARNING] Missing file: {path}")
        return False
    return True

def _exists_dir(path: str) -> bool:
    if not path or not os.path.isdir(path):
        print(f"[WARNING] Missing directory: {path}")
        return False
    return True

def _safe_json_load(path: str) -> Optional[Dict[str, Any]]:
    if not _exists_file(path):
        return None
    try:
        with open(path, "r") as f:
            return json.load(f)
    except Exception as e:
        print(f"[WARNING] Failed to read JSON {path}: {e}")
        return None

def _safe_json_dump(obj: Any, path: str) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        json.dump(obj, f, indent=2, default=str)

def _safe_run(step_name: str, fn, **kwargs) -> Tuple[bool, Optional[Any]]:
    print(f"\n--- {step_name} ---")
    try:
        out = fn(**kwargs)
        print(f"[OK] {step_name}")
        return True, out
    except Exception as e:
        print(f"[ERROR] {step_name}: {e}")
        traceback.print_exc()
        return False, None

def _fmt(x, nd=4):
    if x is None:
        return "N/A"
    try:
        if isinstance(x, (int, float, np.floating)) and np.isnan(x):
            return "N/A"
        return f"{float(x):.{nd}f}"
    except Exception:
        return str(x)

def _run_dir(setting: str, model: str) -> str:
    return os.path.join(OUT_DIR, f"{setting}__{model}")

def _results_json_path(setting: str, model: str) -> str:
    return os.path.join(_run_dir(setting, model), f"results_{setting}_{model}.json")

def _predictions_csv_path(setting: str, model: str) -> str:
    return os.path.join(_run_dir(setting, model), "predictions_test.csv")

def load_stage2_run_index() -> List[Dict[str, Any]]:
    master = os.path.join(OUT_DIR, "all_runs_summary.json")
    data = _safe_json_load(master)
    if isinstance(data, list) and data:
        ok = [r for r in data if r.get("status") == "ok" and r.get("out_dir")]
        print(f"[INFO] Loaded {len(ok)} successful runs from {master}")
        return ok

    runs = []
    if _exists_dir(OUT_DIR):
        for name in os.listdir(OUT_DIR):
            if "__" in name:
                full = os.path.join(OUT_DIR, name)
                if os.path.isdir(full):
                    s, m = name.split("__", 1)
                    runs.append({"setting": s, "model": m, "out_dir": full, "status": "unknown"})
    print(f"[INFO] Scanned {len(runs)} run dirs from OUT_DIR")
    return runs

def get_predictions_jobs(setting: str, model: str) -> List[Dict[str, Any]]:
    res = _safe_json_load(_results_json_path(setting, model))
    jobs = []

    if not res:
        pred = _predictions_csv_path(setting, model)
        if _exists_file(pred):
            jobs.append({
                "tag": f"{setting}__{model}",
                "predictions_csv": pred,
                "threshold": 0.5,
                "output_json": os.path.join(DELAY_DIR, f"{setting}__{model}.json"),
            })
        return jobs

    if setting == "per_platform":
        pred_map = res.get("per_platform_predictions", {}) or {}
        for plat, info in pred_map.items():
            jobs.append({
                "tag": f"{setting}__{model}__{plat}",
                "predictions_csv": info.get("predictions_csv"),
                "threshold": float(info.get("threshold", 0.5)),
                "output_json": os.path.join(DELAY_DIR, f"{setting}__{model}__{plat}.json"),
            })
        return jobs

    # cross_platform
    jobs.append({
        "tag": f"{setting}__{model}",
        "predictions_csv": res.get("predictions_csv") or _predictions_csv_path(setting, model),
        "threshold": float(res.get("threshold", 0.5)),
        "output_json": os.path.join(DELAY_DIR, f"{setting}__{model}.json"),
    })
    return jobs


def _pick_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    cols = set(df.columns)
    for c in candidates:
        if c in cols:
            return c
    return None

def compute_detection_delay_from_predictions(predictions_path: str, threshold: float) -> Dict[str, Any]:
   df = pd.read_csv(predictions_path)
    y_col = _pick_col(df, ["y_true", "label", "target", "y", "gt", "ground_truth"])
    s_col = _pick_col(df, ["y_score", "score", "prob", "proba", "y_pred_proba", "pred_prob", "prediction", "pred"])
    t_col = _pick_col(df, ["AttackTime", "attack_time", "time", "t", "Timestamp", "timestamp"])
    e_col = _pick_col(df, ["Email", "email", "user", "userid", "uid"])

    if y_col is None or s_col is None:
        raise ValueError(
            f"Predictions CSV missing required columns. Found columns: {list(df.columns)}. "
            f"Need y_col in {['y_true','label','target','y']} and s_col in {['y_score','prob','pred_prob','prediction']}."
        )

    y = df[y_col].astype(int).values
    s = df[s_col].astype(float).values
    pred = (s >= float(threshold)).astype(int)

    # time axis
    if t_col is None:
        t = np.arange(len(df), dtype=float)
        time_type = "index"
    else:
        series = df[t_col]
        try:
            tt = pd.to_datetime(series, errors="raise")
            t = tt.view("int64") / 1e9  # seconds
            time_type = "datetime"
        except Exception:
            try:
                t = pd.to_numeric(series, errors="raise").astype(float).values
                time_type = "numeric"
            except Exception:
                t = np.arange(len(df), dtype=float)
                time_type = "index"

    def delay_for_slice(idx: np.ndarray) -> Optional[float]:
        yy = y[idx]
        pp = pred[idx]
        tt = np.asarray(t)[idx]

        pos_idx = np.where(yy == 1)[0]
        if len(pos_idx) == 0:
            return None
        t_first_pos = tt[pos_idx].min()

        tp_idx = np.where((yy == 1) & (pp == 1))[0]
        if len(tp_idx) == 0:
            return None
        t_first_detect = tt[tp_idx].min()

        return float(t_first_detect - t_first_pos)

    delays = []
    detected_count = 0
    total_with_positive = 0

    if e_col is not None:
        for _, g in df.groupby(e_col, sort=False):
            idx = g.index.values
            if (y[idx] == 1).any():
                total_with_positive += 1
                d = delay_for_slice(idx)
                if d is not None:
                    detected_count += 1
                    delays.append(d)
    else:
        if (y == 1).any():
            total_with_positive = 1
            d = delay_for_slice(np.arange(len(df)))
            if d is not None:
                detected_count = 1
                delays.append(d)

    out = {
        "predictions_path": predictions_path,
        "threshold": float(threshold),
        "time_axis": time_type,
        "n_rows": int(len(df)),
        "n_positive_rows": int((y == 1).sum()),
        "n_predicted_positive_rows": int((pred == 1).sum()),
        "n_entities_with_positive": int(total_with_positive),
        "n_entities_detected": int(detected_count),
        "detection_rate_entities": float(detected_count / total_with_positive) if total_with_positive else None,
        "delay_mean": float(np.mean(delays)) if delays else None,
        "delay_median": float(np.median(delays)) if delays else None,
        "delay_p90": float(np.quantile(delays, 0.90)) if delays else None,
        "delay_min": float(np.min(delays)) if delays else None,
        "delay_max": float(np.max(delays)) if delays else None,
    }
    return out

def extract_primary_metrics(results_json: Dict[str, Any], setting: str) -> Dict[str, Any]:
    """
    Supports:
      A) Your updated stage2:
         - cross_platform: results_json["metrics"] = {accuracy,f1,auroc,auprc,threshold,...}
         - per_platform: results_json["metrics"] contains AVERAGES + best_* + n_platforms
      B) Older temp.py-like:
         - metrics_test: {f1, roc_auc, ...}, threshold at top-level
    """
    out: Dict[str, Any] = {"setting": setting}

    # New schema
    if isinstance(results_json.get("metrics"), dict):
        m = results_json["metrics"]
        out.update({
            "accuracy": m.get("accuracy"),
            "f1": m.get("f1"),
            "auroc": m.get("auroc"),
            "auprc": m.get("auprc"),
            "threshold": m.get("threshold", results_json.get("threshold")),
        })
        # per-platform extras (if present)
        out.update({
            "n_platforms": m.get("n_platforms"),
            "best_accuracy": m.get("best_accuracy"),
            "best_accuracy_platform": m.get("best_accuracy_platform"),
            "best_f1": m.get("best_f1"),
            "best_f1_platform": m.get("best_f1_platform"),
            "best_auroc": m.get("best_auroc"),
            "best_auroc_platform": m.get("best_auroc_platform"),
            "best_auprc": m.get("best_auprc"),
            "best_auprc_platform": m.get("best_auprc_platform"),
        })
        return {k: v for k, v in out.items() if v is not None}

    # Old schema
    mtest = results_json.get("metrics_test") or results_json.get("metrics") or {}
    out.update({
        "threshold": results_json.get("threshold"),
        "f1": mtest.get("f1"),
        "auroc": mtest.get("roc_auc") or mtest.get("auroc"),
        "auprc": mtest.get("pr_auc") or mtest.get("auprc"),
        "accuracy": mtest.get("accuracy"),
        "precision": mtest.get("precision"),
        "recall": mtest.get("recall"),
    })
    return {k: v for k, v in out.items() if v is not None}


def compute_cross_platform_benefit(model: str) -> Dict[str, Any]:
    """
    Compare cross_platform vs per_platform for a given model using results JSONs.
    - If per_platform contains averaged metrics, deltas will be computed against those averages.
    """
    per_path = _results_json_path("per_platform", model)
    cross_path = _results_json_path("cross_platform", model)

    per = _safe_json_load(per_path)
    cross = _safe_json_load(cross_path)

    if not per or not cross:
        return {
            "model": model,
            "status": "missing_inputs",
            "per_platform_results_json": per_path,
            "cross_platform_results_json": cross_path,
        }

    per_m = extract_primary_metrics(per, setting="per_platform")
    cross_m = extract_primary_metrics(cross, setting="cross_platform")

    def delta(k):
        if k in per_m and k in cross_m:
            try:
                return float(cross_m[k]) - float(per_m[k])
            except Exception:
                return None
        return None

    benefit = {
        "model": model,
        "status": "ok",
        "per_platform": per_m,
        "cross_platform": cross_m,
        "delta": {
            "accuracy": delta("accuracy"),
            "f1": delta("f1"),
            "auroc": delta("auroc"),
            "auprc": delta("auprc"),
        }
    }
    return benefit

def run_all_detection_delay():
    runs = load_stage2_run_index()
    seen = set()

    for r in runs:
        setting = r.get("setting")
        model = r.get("model")
        if setting not in SETTINGS or model not in MODELS:
            continue
        key = (setting, model)
        if key in seen:
            continue
        seen.add(key)

        jobs = get_predictions_jobs(setting, model)
        if not jobs:
            print(f"[SKIP] No prediction jobs found for {setting}__{model}")
            continue

        for j in jobs:
            pred_csv = j["predictions_csv"]
            if not _exists_file(pred_csv):
                print(f"[SKIP] Missing predictions for {j['tag']}")
                continue

            ok, metrics = _safe_run(
                f"Detection Delay: {j['tag']}",
                compute_detection_delay_from_predictions,
                predictions_path=pred_csv,
                threshold=float(j["threshold"]),
            )
            if ok and metrics is not None:
                _safe_json_dump(metrics, j["output_json"])


def run_all_benefits():
    for m in MODELS:
        out_json = os.path.join(BENEFIT_DIR, f"benefit__{m}.json")
        ok, benefit = _safe_run(f"Cross-Platform Benefit: {m}", compute_cross_platform_benefit, model=m)
        if ok and benefit is not None:
            _safe_json_dump(benefit, out_json)


def generate_summary_report() -> Dict[str, Any]:
    """
    Aggregates:
      - benefits per model
      - delay stats per run (from stage3/delay/*.json)
      - stage2 extracted metrics for table printing
    """
    report: Dict[str, Any] = {"out_dir": OUT_DIR, "models": MODELS, "benefits": {}, "delays": {}, "stage2_metrics": {}}

    # benefits
    for m in MODELS:
        p = os.path.join(BENEFIT_DIR, f"benefit__{m}.json")
        report["benefits"][m] = _safe_json_load(p) or {"status": "missing", "path": p}

    # delays
    if _exists_dir(DELAY_DIR):
        for fn in sorted(os.listdir(DELAY_DIR)):
            if fn.endswith(".json"):
                p = os.path.join(DELAY_DIR, fn)
                report["delays"][fn.replace(".json", "")] = _safe_json_load(p) or {"status": "missing", "path": p}

    # stage2 metrics
    for setting in SETTINGS:
        for m in MODELS:
            p = _results_json_path(setting, m)
            res = _safe_json_load(p)
            key = f"{setting}__{m}"
            if res:
                report["stage2_metrics"][key] = extract_primary_metrics(res, setting=setting)
            else:
                report["stage2_metrics"][key] = {"status": "missing", "path": p}

    # leaderboard by delta f1 if available
    leaderboard = []
    for m, b in report["benefits"].items():
        d = (b.get("delta") or {}).get("f1") if isinstance(b, dict) else None
        if d is not None:
            leaderboard.append((m, d))
    leaderboard.sort(key=lambda x: x[1], reverse=True)
    report["leaderboard_by_delta_f1"] = [{"model": m, "delta_f1": d} for m, d in leaderboard]

    return report


# =============================================================================
# SUMMARY TABLE PRINTING (requested)
# =============================================================================
def _read_delay_for_tag(tag: str, report_delays: Dict[str, Any]) -> Dict[str, Any]:
    d = report_delays.get(tag)
    if not isinstance(d, dict):
        return {}
    return {
        "det_rate_entities": d.get("detection_rate_entities"),
        "delay_mean": d.get("delay_mean"),
        "delay_median": d.get("delay_median"),
        "delay_p90": d.get("delay_p90"),
    }

def _aggregate_per_platform_delay(report_delays: Dict[str, Any], model: str) -> Dict[str, Any]:
    """
    Aggregate per-platform delay JSONs:
      keys look like: per_platform__{model}__{platform}
    Return mean of means for delays, and mean det_rate_entities across platforms.
    """
    prefix = f"per_platform__{model}__"
    rows = []
    for k, v in report_delays.items():
        if k.startswith(prefix) and isinstance(v, dict) and v.get("predictions_path"):
            rows.append(v)

    if not rows:
        return {}

    det_rates = [r.get("detection_rate_entities") for r in rows if r.get("detection_rate_entities") is not None]
    mean_delays = [r.get("delay_mean") for r in rows if r.get("delay_mean") is not None]
    med_delays = [r.get("delay_median") for r in rows if r.get("delay_median") is not None]
    p90_delays = [r.get("delay_p90") for r in rows if r.get("delay_p90") is not None]

    return {
        "det_rate_entities": float(np.mean(det_rates)) if det_rates else None,
        "delay_mean": float(np.mean(mean_delays)) if mean_delays else None,
        "delay_median": float(np.mean(med_delays)) if med_delays else None,
        "delay_p90": float(np.mean(p90_delays)) if p90_delays else None,
        "n_delay_platforms": int(len(rows)),
    }

def print_summary_table(report: Dict[str, Any]) -> pd.DataFrame:
    """
    Prints a clean summary table across:
      - per_platform (averages + best scores if present)
      - cross_platform
    Adds delay metrics:
      - cross_platform uses its delay json directly
      - per_platform uses mean across per-platform delay jsons (if available)
    """
    stage2 = report.get("stage2_metrics", {}) or {}
    delays = report.get("delays", {}) or {}
    benefits = report.get("benefits", {}) or {}

    rows = []
    for setting in SETTINGS:
        for m in MODELS:
            key = f"{setting}__{m}"
            m2 = stage2.get(key, {})
            if not isinstance(m2, dict):
                m2 = {}

            row = {
                "setting": setting,
                "model": m,
                "accuracy": m2.get("accuracy"),
                "f1": m2.get("f1"),
                "auroc": m2.get("auroc"),
                "auprc": m2.get("auprc"),
                "threshold": m2.get("threshold"),
            }

            # per-platform extra columns (from stage2 aggregates, if available)
            row.update({
                "n_platforms_trained": m2.get("n_platforms"),
                "best_accuracy": m2.get("best_accuracy"),
                "best_accuracy_platform": m2.get("best_accuracy_platform"),
                "best_f1": m2.get("best_f1"),
                "best_f1_platform": m2.get("best_f1_platform"),
            })

            # delays
            if setting == "cross_platform":
                row.update(_read_delay_for_tag(key, delays))
            else:
                row.update(_aggregate_per_platform_delay(delays, model=m))

            # benefit deltas (for convenience on each row)
            b = benefits.get(m, {})
            if isinstance(b, dict) and b.get("status") == "ok":
                d = b.get("delta") or {}
                row.update({
                    "delta_accuracy": d.get("accuracy"),
                    "delta_f1": d.get("f1"),
                    "delta_auroc": d.get("auroc"),
                    "delta_auprc": d.get("auprc"),
                })
            else:
                row.update({
                    "delta_accuracy": None,
                    "delta_f1": None,
                    "delta_auroc": None,
                    "delta_auprc": None,
                })

            rows.append(row)

    df = pd.DataFrame(rows)

    # Order rows: cross_platform first within each model, then per_platform
    setting_order = {"cross_platform": 0, "per_platform": 1}
    df["setting_rank"] = df["setting"].map(setting_order).fillna(99).astype(int)
    df = df.sort_values(["model", "setting_rank"], ascending=[True, True]).drop(columns=["setting_rank"])

    # Print
    print("\n" + "=" * 140)
    print("SUMMARY TABLE (Stage2 + Stage3)")
    print(" - cross_platform: metrics from the global model; delays from cross_platform__model delay JSON")
    print(" - per_platform:   metrics are (if available) averaged in Stage2 results; delays are averaged across platforms")
    print("=" * 140)MODELS = ["fta_lstm", "fta_gru", "lstm", "gru", "gnn", "gcn", "gat"] 

    cols = [
        "model", "setting",
        "accuracy", "f1", "auroc", "auprc", "threshold",
        "n_platforms_trained", "best_accuracy", "best_accuracy_platform", "best_f1", "best_f1_platform",
        "det_rate_entities", "delay_mean", "delay_median", "delay_p90",
        "delta_accuracy", "delta_f1", "delta_auroc", "delta_auprc",
    ]

    # ensure columns exist
    for c in cols:
        if c not in df.columns:
            df[c] = None

    # pretty-print with formatting (avoid scientific notation)
    df_print = df[cols].copy()
    for c in ["accuracy","f1","auroc","auprc","threshold","det_rate_entities","delay_mean","delay_median","delay_p90",
              "delta_accuracy","delta_f1","delta_auroc","delta_auprc","best_accuracy","best_f1"]:
        if c in df_print.columns:
            df_print[c] = df_print[c].apply(lambda x: _fmt(x, nd=4))

    print(df_print.to_string(index=False))
    print("\n" + "-" * 140)
    print(f"Saved CSV: {SUMMARY_TABLE_CSV}")
    df.to_csv(SUMMARY_TABLE_CSV, index=False)

    # Quick “best” views
    def _best(df0: pd.DataFrame, setting: str, col: str) -> Optional[pd.Series]:
        sub = df0[df0["setting"] == setting].copy()
        sub = sub[pd.to_numeric(sub[col], errors="coerce").notna()]
        if sub.empty:
            return None
        sub[col] = pd.to_numeric(sub[col], errors="coerce")
        return sub.sort_values(col, ascending=False).iloc[0]

    best_cross_acc = _best(df, "cross_platform", "accuracy")
    best_per_avg_acc = _best(df, "per_platform", "accuracy")

    print("\nBEST RUNS (by Accuracy):")
    if best_cross_acc is not None:
        print(f"  cross_platform best: model={best_cross_acc['model']} ACC={_fmt(best_cross_acc['accuracy'])} F1={_fmt(best_cross_acc['f1'])}")
    else:
        print("  cross_platform best: N/A")

    if best_per_avg_acc is not None:
        print(f"  per_platform best (avg): model={best_per_avg_acc['model']} ACC={_fmt(best_per_avg_acc['accuracy'])} F1={_fmt(best_per_avg_acc['f1'])}")
        if pd.notna(best_per_avg_acc.get("best_accuracy_platform")):
            print(f"    best platform for that model: {best_per_avg_acc.get('best_accuracy_platform')} "
                  f"(best_acc={_fmt(best_per_avg_acc.get('best_accuracy'))})")
    else:
        print("  per_platform best (avg): N/A")

    return df


# =============================================================================
# MAIN
# =============================================================================
def main():
    print("=" * 80)
    print("STAGE 3 (SELF-CONTAINED) — updated for models:")
    print("  " + ", ".join(MODELS))
    print(f"Stage2 OUT_DIR: {OUT_DIR}")
    print(f"Stage3 OUT:     {STAGE3_DIR}")
    print("=" * 80)

    run_all_detection_delay()
    run_all_benefits()

    ok, rep = _safe_run("Summary Report", generate_summary_report)
    if ok and rep is not None:
        _safe_json_dump(rep, REPORT_PATH)
        print(f"[OK] Saved report: {REPORT_PATH}")

        # print the requested summary table
        _ = print_summary_table(rep)


if __name__ == "__main__":
    main()


# Platforms Detection Modelling

In [ ]:
#!/usr/bin/env python3
"""
GPU-Optimized Time-Series Classification Models for Platform Detection

Handles:
- LSTM classifier (PyTorch with CUDA)
- GRU classifier (PyTorch with CUDA)
- FTA-GRU simplified classifier (PyTorch with CUDA)
- FTA-LSTM simplified classifier (PyTorch with CUDA)
- Temporal GNN classifier (timesteps-as-nodes; PyTorch with CUDA)
- IFCNN-TPP classifier (multi-kernel temporal CNN + mean pooling; PyTorch with CUDA)   <-- UPDATED
- CEP3 classifier (dilated temporal convolution encoder; PyTorch with CUDA)
"""

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    f1_score, classification_report, confusion_matrix,
    roc_auc_score, roc_curve, auc
)
from sklearn.preprocessing import label_binarize
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


# =============================================================================
# DEVICE
# =============================================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    print(f"GPU Memory: {props.total_memory / 1024**3:.2f} GB")


# =============================================================================
# BASIC MODELS (LSTM, GRU)
# =============================================================================

class LSTMClassifier(nn.Module):
    """LSTM-based time-series classifier"""
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)         # (num_layers, B, H)
        out = self.dropout(hidden[-1])        # (B, H)
        return self.fc(out)                   # (B, C)


class GRUClassifier(nn.Module):
    """GRU-based time-series classifier"""
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, hidden = self.gru(x)               # (num_layers, B, H)
        out = self.dropout(hidden[-1])        # (B, H)
        return self.fc(out)                   # (B, C)


# =============================================================================
# FTA ENCODERS (FTA-GRU, FTA-LSTM)
# =============================================================================

class FTAGru(nn.Module):
    """
    FTA-GRU:
    - Feature-level attention (gate per feature per timestep)
    - Multi-layer GRU
    - Multi-head temporal attention
    - Attention pooling
    """
    def __init__(self, input_size, hidden_size, num_classes,
                 num_gru_layers=2, attn_dim=64, num_attn_heads=4, dropout=0.3):
        super().__init__()
        self.feature_attention = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.Tanh(),
            nn.Linear(input_size, input_size),
            nn.Sigmoid()
        )
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_gru_layers,
            batch_first=True,
            dropout=dropout if num_gru_layers > 1 else 0.0,
            bidirectional=False
        )
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=num_attn_heads,
            dropout=dropout,
            batch_first=True
        )
        self.attn_pooling = nn.Sequential(
            nn.Linear(hidden_size, attn_dim),
            nn.Tanh(),
            nn.Linear(attn_dim, 1)
        )
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes)
        )

    def forward(self, x):
        feat_w = self.feature_attention(x)
        x = x * feat_w
        gru_out, _ = self.gru(x)
        attn_out, _ = self.temporal_attention(gru_out, gru_out, gru_out)
        scores = self.attn_pooling(attn_out).squeeze(-1)
        weights = F.softmax(scores, dim=1).unsqueeze(-1)
        pooled = (attn_out * weights).sum(dim=1)
        pooled = self.dropout(pooled)
        return self.classifier(pooled)


class FTALstm(nn.Module):
    """
    FTA-LSTM:
    - Feature-level attention
    - Multi-layer LSTM
    - Multi-head temporal attention
    - Attention pooling
    """
    def __init__(self, input_size, hidden_size, num_classes,
                 num_lstm_layers=2, attn_dim=64, num_attn_heads=4, dropout=0.3):
        super().__init__()
        self.feature_attention = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.Tanh(),
            nn.Linear(input_size, input_size),
            nn.Sigmoid()
        )
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_lstm_layers,
            batch_first=True,
            dropout=dropout if num_lstm_layers > 1 else 0.0,
            bidirectional=False
        )
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=num_attn_heads,
            dropout=dropout,
            batch_first=True
        )
        self.attn_pooling = nn.Sequential(
            nn.Linear(hidden_size, attn_dim),
            nn.Tanh(),
            nn.Linear(attn_dim, 1)
        )
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes)
        )

    def forward(self, x):
        feat_w = self.feature_attention(x)
        x = x * feat_w
        lstm_out, _ = self.lstm(x)
        attn_out, _ = self.temporal_attention(lstm_out, lstm_out, lstm_out)
        scores = self.attn_pooling(attn_out).squeeze(-1)
        weights = F.softmax(scores, dim=1).unsqueeze(-1)
        pooled = (attn_out * weights).sum(dim=1)
        pooled = self.dropout(pooled)
        return self.classifier(pooled)


# =============================================================================
# TEMPORAL GNN (timesteps-as-nodes)
# =============================================================================

class GraphAttentionLayer(nn.Module):
    """Dense graph-attention across timesteps (treat T as nodes)."""
    def __init__(self, in_features, out_features, dropout=0.2, alpha=0.2):
        super().__init__()
        self.dropout = dropout
        self.alpha = alpha

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)

        self.a = nn.Parameter(torch.zeros(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, x, adj=None):
        B, T, _ = x.size()
        h = torch.matmul(x, self.W)  # (B, T, Fout)

        h_repeat_1 = h.repeat_interleave(T, dim=1)
        h_repeat_2 = h.repeat(1, T, 1)
        h_concat = torch.cat([h_repeat_1, h_repeat_2], dim=-1)

        e = self.leakyrelu(torch.matmul(h_concat, self.a)).view(B, T, T)

        if adj is not None:
            e = e.masked_fill(adj == 0, -1e9)

        attention = F.softmax(e, dim=-1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        return torch.matmul(attention, h)


class GNNClassifier(nn.Module):
    """
    Temporal GNN:
    - GRU encodes per-timestep hidden states
    - GAT across timesteps (dense)
    - Multihead temporal attention
    - Mean pooling -> classifier
    """
    def __init__(self, input_size, hidden_size, num_classes, num_layers=2, gat_heads=4, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )

        self.gat_layers = nn.ModuleList([
            GraphAttentionLayer(hidden_size, hidden_size // gat_heads, dropout=dropout)
            for _ in range(gat_heads)
        ])

        self.layer_norm = nn.LayerNorm(hidden_size)

        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=4,
            dropout=dropout,
            batch_first=True
        )

        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        gru_out, _ = self.gru(x)  # (B, T, H)
        gat_outputs = [gat(gru_out) for gat in self.gat_layers]
        gat_combined = torch.cat(gat_outputs, dim=-1)  # (B, T, H)

        gru_out = self.layer_norm(gru_out + gat_combined)
        attn_out, _ = self.temporal_attention(gru_out, gru_out, gru_out)
        pooled = attn_out.mean(dim=1)

        out = self.dropout(pooled)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        return self.fc2(out)


# =============================================================================
# IFCNN-TPP (UPDATED: multi-kernel temporal CNN + mean pooling)
# =============================================================================

class IFCNNTPPClassifier(nn.Module):
    """
    ifcnn_tpp (multi-kernel temporal CNN proxy):
      - Linear projection: F -> H
      - Parallel Conv1d branches over time with different kernel sizes (e.g., 3/5/7)
      - Average branch outputs
      - Mean pool over time
      - MLP head -> logits

    Input:  x (B, T, F)
    Conv1d: expects (B, H, T)
    """
    def __init__(
        self,
        input_size: int,
        hidden_size: int,
        num_classes: int,
        kernels=(3, 5, 7),
        dropout: float = 0.3,
    ):
        super().__init__()
        self.in_proj = nn.Linear(input_size, hidden_size)

        self.convs = nn.ModuleList([
            nn.Conv1d(hidden_size, hidden_size, kernel_size=k, padding=k // 2)
            for k in kernels
        ])

        self.norm = nn.LayerNorm(hidden_size)

        self.head = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes),
        )

    def forward(self, x):
        # x: (B, T, F) -> h: (B, T, H)
        h = self.norm(self.in_proj(x))

        # Conv over time: (B, H, T)
        h = h.transpose(1, 2)

        outs = [F.relu(conv(h)) for conv in self.convs]           # list of (B, H, T)
        z = torch.stack(outs, dim=0).mean(dim=0)                  # (B, H, T)

        # back to (B, T, H) then mean pool over time -> (B, H)
        z = z.transpose(1, 2)
        pooled = z.mean(dim=1)

        return self.head(pooled)


# =============================================================================
# CEP3 (Dilated Temporal Convolution Encoder)
# =============================================================================

class CEP3Classifier(nn.Module):
    """
    CEP3-like dilated TCN:
      - Stacked dilated Conv1d blocks with residual connections
      - Global average pooling over time
      - Classification head

    Input: (B, T, F)
      Conv1d works on (B, F, T)
    """
    def __init__(
        self,
        input_size: int,
        num_classes: int,
        channels: int = 128,
        kernel_size: int = 3,
        n_blocks: int = 4,
        dropout: float = 0.3,
    ):
        super().__init__()
        assert kernel_size % 2 == 1, "Use odd kernel_size for simple same-length padding."

        self.in_proj = nn.Conv1d(input_size, channels, kernel_size=1)

        blocks = []
        for i in range(n_blocks):
            dilation = 2 ** i
            pad = (kernel_size // 2) * dilation

            blocks.append(nn.Sequential(
                nn.Conv1d(channels, channels, kernel_size=kernel_size, padding=pad, dilation=dilation),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Conv1d(channels, channels, kernel_size=kernel_size, padding=pad, dilation=dilation),
                nn.ReLU(),
                nn.Dropout(dropout),
            ))
        self.blocks = nn.ModuleList(blocks)

        self.out_norm = nn.LayerNorm(channels)
        self.fc = nn.Linear(channels, num_classes)

    def forward(self, x):
        # x: (B, T, F) -> (B, F, T)
        x = x.transpose(1, 2)
        h = self.in_proj(x)  # (B, C, T)

        # residual dilated blocks (length preserved by padding choice)
        for blk in self.blocks:
            r = h
            h = blk(h)
            h = h + r

        # global average pool over time: (B, C)
        pooled = h.mean(dim=-1)          # (B, C)
        pooled = self.out_norm(pooled)   # LayerNorm over channels
        return self.fc(pooled)


# =============================================================================
# TRAINING UTILITIES
# =============================================================================

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, pred, target):
        n_classes = pred.size(-1)
        log_probs = F.log_softmax(pred, dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(log_probs)
            true_dist.fill_(self.smoothing / (n_classes - 1))
            true_dist.scatter_(1, target.unsqueeze(1), 1.0 - self.smoothing)
        return torch.mean(torch.sum(-true_dist * log_probs, dim=-1))


class TimeSeriesAugmentation:
    @staticmethod
    def jitter(x, sigma=0.03):
        noise = np.random.normal(0, sigma, x.shape).astype(np.float32)
        return x + noise

    @staticmethod
    def scaling(x, sigma=0.1):
        factor = np.random.normal(1.0, sigma, (x.shape[0], 1, x.shape[2])).astype(np.float32)
        return x * factor


def augment_batch(X_batch, y_batch, aug_prob=0.3):
    if np.random.random() < aug_prob:
        choice = np.random.choice(['jitter', 'scaling'])
        X_aug = X_batch.detach().cpu().numpy()
        X_aug = TimeSeriesAugmentation.jitter(X_aug) if choice == 'jitter' else TimeSeriesAugmentation.scaling(X_aug)
        return torch.FloatTensor(X_aug).to(X_batch.device), y_batch
    return X_batch, y_batch


# =============================================================================
# DATA LOADING AND PREPARATION
# =============================================================================

def load_dataset(npz_path):
    print(f"\nLoading dataset from {npz_path}...")
    data = np.load(npz_path, allow_pickle=True)

    X = data['X']
    y_platform = data['y_platform']
    y_time_bucket = data['y_time_bucket']
    platform_classes = data['platform_classes']

    print(f"X shape: {X.shape}")
    print(f"Number of platform classes: {len(platform_classes)}")
    print(f"Example classes: {platform_classes[:5]}")
    return X, y_platform, y_time_bucket, platform_classes


def prepare_dataloaders(X, y, batch_size=64, train_ratio=0.7, val_ratio=0.15):
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=(1 - train_ratio - val_ratio), random_state=42, stratify=y
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp,
        test_size=val_ratio / (train_ratio + val_ratio),
        random_state=42,
        stratify=y_temp
    )

    print(f"\nDataset splits:")
    print(f"  Train: {X_train.shape[0]} samples")
    print(f"  Val:   {X_val.shape[0]} samples")
    print(f"  Test:  {X_test.shape[0]} samples")

    X_train_t = torch.FloatTensor(X_train).to(device)
    y_train_t = torch.LongTensor(y_train).to(device)
    X_val_t = torch.FloatTensor(X_val).to(device)
    y_val_t = torch.LongTensor(y_val).to(device)
    X_test_t = torch.FloatTensor(X_test).to(device)
    y_test_t = torch.LongTensor(y_test).to(device)

    train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(TensorDataset(X_val_t, y_val_t),     batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(TensorDataset(X_test_t, y_test_t),   batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader


# =============================================================================
# TRAINING AND EVALUATION
# =============================================================================

def train_epoch(model, train_loader, criterion, optimizer, use_augmentation=False, aug_prob=0.3):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        if use_augmentation:
            X_batch, y_batch = augment_batch(X_batch, y_batch, aug_prob)

        optimizer.zero_grad(set_to_none=True)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += float(loss.item())
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

    return total_loss / max(1, len(train_loader)), 100.0 * correct / max(1, total)


def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += float(loss.item())

            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)

            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

            all_preds.extend(predicted.detach().cpu().numpy().tolist())
            all_labels.extend(y_batch.detach().cpu().numpy().tolist())
            all_probs.extend(probs.detach().cpu().numpy().tolist())

    return (
        total_loss / max(1, len(data_loader)),
        100.0 * correct / max(1, total),
        np.array(all_preds),
        np.array(all_labels),
        np.array(all_probs),
    )


def train_model(model, train_loader, val_loader, num_epochs=200, learning_rate=1e-3,
                patience=10, use_label_smoothing=True, use_augmentation=True):
    if use_label_smoothing:
        criterion = LabelSmoothingCrossEntropy(smoothing=0.1)
        print("Using Label Smoothing (0.1)")
    else:
        criterion = nn.CrossEntropyLoss()

    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=10, T_mult=2, eta_min=1e-6
    )

    best_val_loss = float('inf')
    epochs_without_improvement = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_path = '/tmp/best_model.pth'

    print("\nTraining started...")
    print(f"{'Epoch':>5} | {'Train Loss':>12} | {'Train Acc':>10} | {'Val Loss':>12} | {'Val Acc':>10}")
    print("-" * 70)

    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer,
            use_augmentation=use_augmentation, aug_prob=0.3
        )
        val_loss, val_acc, _, _, _ = evaluate(model, val_loader, criterion)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        scheduler.step()

        print(f"{epoch+1:5d} | {train_loss:12.4f} | {train_acc:9.2f}% | {val_loss:12.4f} | {val_acc:9.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_path)
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

    model.load_state_dict(torch.load(best_path, map_location=device))
    return model, history


def compute_metrics(y_true, y_pred, y_probs, class_names):
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    f1_per_class = f1_score(y_true, y_pred, average=None)

    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)

    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    if len(class_names) > 2:
        roc_auc = roc_auc_score(y_true_bin, y_probs, average='macro', multi_class='ovr')
    else:
        roc_auc = roc_auc_score(y_true, y_probs[:, 1])

    return {
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'f1_per_class': dict(zip(class_names, f1_per_class)),
        'roc_auc': roc_auc,
        'confusion_matrix': cm,
        'classification_report': report
    }


# =============================================================================
# VISUALIZATION (unchanged)
# =============================================================================

def plot_training_history(history, output_path):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].plot(history['train_loss'], label='Train Loss')
    axes[0].plot(history['val_loss'], label='Val Loss')
    axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss'); axes[0].legend(); axes[0].grid(True)

    axes[1].plot(history['train_acc'], label='Train Acc')
    axes[1].plot(history['val_acc'], label='Val Acc')
    axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title('Training and Validation Accuracy'); axes[1].legend(); axes[1].grid(True)

    plt.tight_layout()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Training history saved to {output_path}")


def plot_confusion_matrix(cm, class_names, output_path):
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=False, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('True', fontsize=12)
    plt.title('Confusion Matrix', fontsize=14)
    plt.xticks(rotation=45, ha='right'); plt.yticks(rotation=0)
    plt.tight_layout()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Confusion matrix saved to {output_path}")


def plot_roc_curves(y_true, y_probs, class_names, output_path):
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    plt.figure(figsize=(12, 10))
    colors = plt.cm.tab20(np.linspace(0, 1, len(class_names)))

    for i, (class_name, color) in enumerate(zip(class_names, colors)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_name} (AUC = {roc_auc:.3f})', color=color, linewidth=2)

    plt.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=2)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves (One-vs-Rest)', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"ROC curves saved to {output_path}")


# =============================================================================
# MAIN TRAINING PIPELINE
# =============================================================================

def main():
    OUTPUT_DIR = '/home/Passwords/Analysis/Platform_Output'
    DATA_PATH = '/home/Passwords/Analysis/platforms.npz'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    X, y_platform, y_time_bucket, platform_classes = load_dataset(DATA_PATH)
    train_loader, val_loader, test_loader = prepare_dataloaders(X, y_platform, batch_size=64)

    input_size = X.shape[2]
    num_classes = len(platform_classes)

    print("\n" + "=" * 70)
    print("TRAINING MODELS")
    print("=" * 70)
    print(f"Input size: {input_size}")
    print(f"Number of classes: {num_classes}")

    models_to_train = {
        'LSTM': LSTMClassifier(input_size, 128, 2, num_classes, dropout=0.3).to(device),
        'GRU': GRUClassifier(input_size, 128, 2, num_classes, dropout=0.3).to(device),
        'FTA-GRU': FTAGru(input_size, 128, num_classes, num_gru_layers=2, attn_dim=64, num_attn_heads=4, dropout=0.3).to(device),
        'FTA-LSTM': FTALstm(input_size, 128, num_classes, num_lstm_layers=2, attn_dim=64, num_attn_heads=4, dropout=0.3).to(device),
        'GNN': GNNClassifier(input_size, 128, num_classes, num_layers=2, gat_heads=4, dropout=0.3).to(device),

        # UPDATED: multi-kernel CNN proxy (no GRU/LSTM encoder)
        'IFCNN-TPP': IFCNNTPPClassifier(
            input_size=input_size,
            hidden_size=128,
            num_classes=num_classes,
            kernels=(3, 5, 7),
            dropout=0.3,
        ).to(device),

        'CEP3': CEP3Classifier(
            input_size=input_size,
            num_classes=num_classes,
            channels=128,
            kernel_size=3,
            n_blocks=4,
            dropout=0.3,
        ).to(device),
    }

    results = {}

    for model_name, model in models_to_train.items():
        print(f"\n{'=' * 70}")
        print(f"Training {model_name} Model")
        print(f"{'=' * 70}")

        num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Number of parameters: {num_params:,}")

        # Treat attention/CNN/TCN as "advanced"
        use_advanced = model_name in ['FTA-GRU', 'FTA-LSTM', 'IFCNN-TPP', 'CEP3']

        trained_model, history = train_model(
            model, train_loader, val_loader,
            num_epochs=200,
            learning_rate=0.001,
            patience=10,
            use_label_smoothing=use_advanced,
            use_augmentation=use_advanced
        )

        print(f"\nEvaluating {model_name} on test set...")
        criterion = nn.CrossEntropyLoss()
        test_loss, test_acc, y_pred, y_true, y_probs = evaluate(trained_model, test_loader, criterion)

        print(f"Test Loss: {test_loss:.4f}")
        print(f"Test Accuracy: {test_acc:.2f}%")

        metrics = compute_metrics(y_true, y_pred, y_probs, platform_classes)

        print(f"\nF1 Score (Macro): {metrics['f1_macro']:.4f}")
        print(f"F1 Score (Weighted): {metrics['f1_weighted']:.4f}")
        print(f"ROC-AUC: {metrics['roc_auc']:.4f}")

        results[model_name] = {
            'model': trained_model,
            'history': history,
            'test_metrics': metrics,
            'test_accuracy': test_acc
        }

        plot_training_history(history, f'{OUTPUT_DIR}/{model_name}_training_history.png')
        plot_confusion_matrix(metrics['confusion_matrix'], platform_classes, f'{OUTPUT_DIR}/{model_name}_confusion_matrix.png')
        plot_roc_curves(y_true, y_probs, platform_classes, f'{OUTPUT_DIR}/{model_name}_roc_curves.png')

        torch.save(trained_model.state_dict(), f'{OUTPUT_DIR}/{model_name}_model.pth')
        print(f"\n{model_name} training complete!")

    print("\n" + "=" * 70)
    print("MODEL COMPARISON")
    print("=" * 70)

    comparison_df = pd.DataFrame({
        'Model': list(results.keys()),
        'Test Accuracy (%)': [r['test_accuracy'] for r in results.values()],
        'F1 Macro': [r['test_metrics']['f1_macro'] for r in results.values()],
        'F1 Weighted': [r['test_metrics']['f1_weighted'] for r in results.values()],
        'ROC-AUC': [r['test_metrics']['roc_auc'] for r in results.values()]
    }).sort_values('Test Accuracy (%)', ascending=False)

    print("\n" + comparison_df.to_string(index=False))
    comparison_df.to_csv(f'{OUTPUT_DIR}/model_comparison.csv', index=False)

    # Comparison chart
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    metrics_to_plot = [
        ('Test Accuracy (%)', axes[0, 0]),
        ('F1 Macro', axes[0, 1]),
        ('F1 Weighted', axes[1, 0]),
        ('ROC-AUC', axes[1, 1])
    ]

    for metric_name, ax in metrics_to_plot:
        data = comparison_df.sort_values(metric_name, ascending=True)
        colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(data)))
        ax.barh(data['Model'], data[metric_name], color=colors)
        ax.set_xlabel(metric_name, fontsize=11)
        ax.set_title(f'{metric_name} Comparison', fontsize=12, fontweight='bold')
        ax.grid(axis='x', alpha=0.3)
        for i, v in enumerate(data[metric_name]):
            ax.text(v, i, f' {v:.3f}', va='center', fontsize=10)

    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/model_comparison_chart.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Comparison chart saved to {OUTPUT_DIR}/model_comparison_chart.png")

    best_model = comparison_df.iloc[0]['Model']
    best_acc = comparison_df.iloc[0]['Test Accuracy (%)']
    print(f"\n{'=' * 70}")
    print(f"BEST MODEL: {best_model} with {best_acc:.2f}% Test Accuracy")
    print(f"{'=' * 70}")

    print("\nGenerated files:")
    print("  - *_model.pth: Trained model weights")
    print("  - *_training_history.png: Training curves")
    print("  - *_confusion_matrix.png: Confusion matrices")
    print("  - *_roc_curves.png: ROC curves")
    print("  - model_comparison.csv: Performance comparison table")
    print("  - model_comparison_chart.png: Visual comparison")


if __name__ == "__main__":
    main()


## Detection Rate and delay for Platform detection

In [ ]:
#!/usr/bin/env python3
"""
Time-to-Detection Analysis for Cross-Platform Breach Platform Classification

Measures how quickly we can correctly identify the breach platform over time.

Models:
- LSTM classifier (PyTorch with CUDA)
- GRU classifier (PyTorch with CUDA)
- FTA-GRU simplified classifier (PyTorch with CUDA)
- FTA-LSTM simplified classifier (PyTorch with CUDA)
- Temporal GNN classifier (timesteps-as-nodes; PyTorch with CUDA)
- IFCNN-TPP classifier (Multi-kernel CNN; PyTorch with CUDA)   <-- UPDATED
- CEP3 classifier (dilated temporal convolution encoder; PyTorch with CUDA)
"""

import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ============================================================================
# MODEL ARCHITECTURES (must match training code)
# ============================================================================

class LSTMClassifier(nn.Module):
    """LSTM-based time-series classifier"""
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)   # hidden: (num_layers, B, H)
        out = self.dropout(hidden[-1])  # (B, H)
        return self.fc(out)             # (B, C)


class GRUClassifier(nn.Module):
    """GRU-based time-series classifier"""
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, hidden = self.gru(x)         # hidden: (num_layers, B, H)
        out = self.dropout(hidden[-1])  # (B, H)
        return self.fc(out)             # (B, C)


class FTAGru(nn.Module):
    """FTA-GRU: Feature-Temporal Attention GRU"""
    def __init__(self, input_size, hidden_size, num_classes,
                 num_gru_layers=2, attn_dim=64, num_attn_heads=4, dropout=0.3):
        super().__init__()
        self.feature_attention = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.Tanh(),
            nn.Linear(input_size, input_size),
            nn.Sigmoid()
        )
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_gru_layers,
            batch_first=True,
            dropout=dropout if num_gru_layers > 1 else 0.0,
            bidirectional=False
        )
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=num_attn_heads,
            dropout=dropout,
            batch_first=True
        )
        self.attn_pooling = nn.Sequential(
            nn.Linear(hidden_size, attn_dim),
            nn.Tanh(),
            nn.Linear(attn_dim, 1)
        )
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes)
        )

    def forward(self, x):
        feat_w = self.feature_attention(x)
        x = x * feat_w
        gru_out, _ = self.gru(x)
        attn_out, _ = self.temporal_attention(gru_out, gru_out, gru_out)
        scores = self.attn_pooling(attn_out).squeeze(-1)
        weights = F.softmax(scores, dim=1).unsqueeze(-1)
        pooled = (attn_out * weights).sum(dim=1)
        pooled = self.dropout(pooled)
        return self.classifier(pooled)


class FTALstm(nn.Module):
    """FTA-LSTM: Feature-Temporal Attention LSTM"""
    def __init__(self, input_size, hidden_size, num_classes,
                 num_lstm_layers=2, attn_dim=64, num_attn_heads=4, dropout=0.3):
        super().__init__()
        self.feature_attention = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.Tanh(),
            nn.Linear(input_size, input_size),
            nn.Sigmoid()
        )
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_lstm_layers,
            batch_first=True,
            dropout=dropout if num_lstm_layers > 1 else 0.0,
            bidirectional=False
        )
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=num_attn_heads,
            dropout=dropout,
            batch_first=True
        )
        self.attn_pooling = nn.Sequential(
            nn.Linear(hidden_size, attn_dim),
            nn.Tanh(),
            nn.Linear(attn_dim, 1)
        )
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes)
        )

    def forward(self, x):
        feat_w = self.feature_attention(x)
        x = x * feat_w
        lstm_out, _ = self.lstm(x)
        attn_out, _ = self.temporal_attention(lstm_out, lstm_out, lstm_out)
        scores = self.attn_pooling(attn_out).squeeze(-1)
        weights = F.softmax(scores, dim=1).unsqueeze(-1)
        pooled = (attn_out * weights).sum(dim=1)
        pooled = self.dropout(pooled)
        return self.classifier(pooled)


class GraphAttentionLayer(nn.Module):
    """Graph Attention Layer for dense timesteps-as-nodes graph"""
    def __init__(self, in_features, out_features, dropout=0.2, alpha=0.2):
        super().__init__()
        self.dropout = dropout

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)

        self.a = nn.Parameter(torch.zeros(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(alpha)

    def forward(self, x, adj=None):
        B, T, _ = x.size()
        h = torch.matmul(x, self.W)  # (B,T,Fout)

        h1 = h.repeat_interleave(T, dim=1)
        h2 = h.repeat(1, T, 1)
        hcat = torch.cat([h1, h2], dim=-1)

        e = self.leakyrelu(torch.matmul(hcat, self.a)).view(B, T, T)

        if adj is not None:
            e = e.masked_fill(adj == 0, -1e9)

        att = F.softmax(e, dim=-1)
        att = F.dropout(att, self.dropout, training=self.training)
        return torch.matmul(att, h)


class GNNClassifier(nn.Module):
    """Temporal GNN: GRU -> GAT across timesteps -> temporal attention -> pool -> classify"""
    def __init__(self, input_size, hidden_size, num_classes, num_layers=2, gat_heads=4, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.gat_layers = nn.ModuleList([
            GraphAttentionLayer(hidden_size, hidden_size // gat_heads, dropout=dropout)
            for _ in range(gat_heads)
        ])
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=4,
            dropout=dropout,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        gru_out, _ = self.gru(x)  # (B,T,H)
        gat_outs = [gat(gru_out) for gat in self.gat_layers]
        gat_combined = torch.cat(gat_outs, dim=-1)  # (B,T,H)
        h = self.layer_norm(gru_out + gat_combined)
        attn_out, _ = self.temporal_attention(h, h, h)
        pooled = attn_out.mean(dim=1)
        out = self.dropout(pooled)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        return self.fc2(out)


# ----------------------------------------------------------------------------
# UPDATED: IFCNN-TPP (code-1 multi-kernel CNN version)
# ----------------------------------------------------------------------------

class IFCNNTPPClassifier(nn.Module):
    """
    IFCNN-TPP (Multi-kernel CNN, code-1 style):
      - in_proj: Linear(F -> H)
      - convs: multi-kernel Conv1d over time on hidden channels
      - mean of conv branches
      - mean pool over time
      - head MLP

    NOTE: This matches checkpoints with keys like:
      in_proj.*, convs.*, norm.*, head.*
    """
    def __init__(self, input_size, hidden_size, num_classes, kernels=(3, 5, 7), dropout=0.3):
        super().__init__()
        self.in_proj = nn.Linear(input_size, hidden_size)
        self.convs = nn.ModuleList([
            nn.Conv1d(hidden_size, hidden_size, kernel_size=k, padding=k // 2)
            for k in kernels
        ])
        self.norm = nn.LayerNorm(hidden_size)
        self.head = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes),
        )

    def forward(self, x):
        # x: (B,T,F)
        h = self.norm(self.in_proj(x))  # (B,T,H)
        h = h.transpose(1, 2)           # (B,H,T)

        outs = [F.relu(conv(h)) for conv in self.convs]   # each (B,H,T)
        z = torch.stack(outs, dim=0).mean(dim=0)          # (B,H,T)

        z = z.transpose(1, 2)       # (B,T,H)
        pooled = z.mean(dim=1)      # (B,H)
        return self.head(pooled)    # (B,C)


# ----------------------------------------------------------------------------
# CEP3 (dilated temporal convolution encoder)
# ----------------------------------------------------------------------------

class CEP3Classifier(nn.Module):
    """
    CEP3-like dilated TCN with residual blocks.
    Input: (B, T, F)
    """
    def __init__(
        self,
        input_size: int,
        num_classes: int,
        channels: int = 128,
        kernel_size: int = 3,
        n_blocks: int = 4,
        dropout: float = 0.3,
    ):
        super().__init__()
        assert kernel_size % 2 == 1, "Use odd kernel_size."

        self.in_proj = nn.Conv1d(input_size, channels, kernel_size=1)
        self.blocks = nn.ModuleList()

        for i in range(n_blocks):
            dilation = 2 ** i
            pad = (kernel_size // 2) * dilation
            self.blocks.append(nn.Sequential(
                nn.Conv1d(channels, channels, kernel_size=kernel_size, padding=pad, dilation=dilation),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Conv1d(channels, channels, kernel_size=kernel_size, padding=pad, dilation=dilation),
                nn.ReLU(),
                nn.Dropout(dropout),
            ))

        self.out_norm = nn.LayerNorm(channels)
        self.fc = nn.Linear(channels, num_classes)

    def forward(self, x):
        # (B,T,F) -> (B,F,T)
        x = x.transpose(1, 2)
        h = self.in_proj(x)  # (B,C,T)

        for blk in self.blocks:
            r = h
            h = blk(h)
            h = h + r

        pooled = h.mean(dim=-1)  # (B,C)
        pooled = self.out_norm(pooled)
        return self.fc(pooled)


# ============================================================================
# DATA LOADING AND MODEL LOADING
# ============================================================================

def load_dataset(npz_path):
    print(f"\nLoading dataset from {npz_path}...")
    data = np.load(npz_path, allow_pickle=True)

    X = data["X"]
    y_platform = data["y_platform"]
    platform_classes = data["platform_classes"]

    print(f"X shape: {X.shape}")
    print(f"Number of platform classes: {len(platform_classes)}")
    return X, y_platform, platform_classes


def load_trained_model(model_type, input_size, num_classes, model_path):
    """
    Load a trained model (must match the training script’s architecture/hparams).

    Includes a safe auto-detect for IFCNN-TPP checkpoints to avoid key mismatches.
    """
    state = torch.load(model_path, map_location=device)

    if model_type == "LSTM":
        model = LSTMClassifier(input_size, 128, 2, num_classes, dropout=0.3)

    elif model_type == "GRU":
        model = GRUClassifier(input_size, 128, 2, num_classes, dropout=0.3)

    elif model_type == "FTA-GRU":
        model = FTAGru(
            input_size=input_size,
            hidden_size=128,
            num_classes=num_classes,
            num_gru_layers=2,
            attn_dim=64,
            num_attn_heads=4,
            dropout=0.3,
        )

    elif model_type == "FTA-LSTM":
        model = FTALstm(
            input_size=input_size,
            hidden_size=128,
            num_classes=num_classes,
            num_lstm_layers=2,
            attn_dim=64,
            num_attn_heads=4,
            dropout=0.3,
        )

    elif model_type == "GNN":
        model = GNNClassifier(
            input_size=input_size,
            hidden_size=128,
            num_classes=num_classes,
            num_layers=2,
            gat_heads=4,
            dropout=0.3,
        )

    elif model_type == "IFCNN-TPP":
        # Expecting code-1 multi-kernel checkpoint keys:
        # in_proj.*, convs.*, norm.*, head.*
        if any(k.startswith("in_proj.") for k in state.keys()):
            model = IFCNNTPPClassifier(
                input_size=input_size,
                hidden_size=128,
                num_classes=num_classes,
                kernels=(3, 5, 7),
                dropout=0.3,
            )
        else:
            raise RuntimeError(
                "IFCNN-TPP checkpoint does not look like the multi-kernel code-1 variant. "
                "Expected keys like 'in_proj.*', 'convs.*', 'norm.*', 'head.*'."
            )

    elif model_type == "CEP3":
        model = CEP3Classifier(
            input_size=input_size,
            num_classes=num_classes,
            channels=128,
            kernel_size=3,
            n_blocks=4,
            dropout=0.3,
        )

    else:
        raise ValueError(f"Unknown model type: {model_type}")

    model.load_state_dict(state)
    model.to(device)
    model.eval()
    return model


# ============================================================================
# TIME-TO-DETECTION ANALYSIS
# ============================================================================

def compute_detection_time(model, X, y_true, confidence_threshold=0.7):
    """
    Detection happens at time step t when:
      1) predicted platform == true platform
      2) confidence(pred) >= threshold

    We evaluate partial prefixes by padding to full seq_len (so all models keep same input shape).
    """
    detection_times = []
    detection_confidences = []

    seq_len = X.shape[1]
    feat_dim = X.shape[2]

    for i in range(len(X)):
        sequence = X[i]  # (seq_len, feat_dim)
        true_label = int(y_true[i])

        detection_time = -1
        detection_conf = 0.0

        for t in range(1, seq_len + 1):
            partial = sequence[:t]  # (t, feat_dim)

            if t < seq_len:
                padding = np.zeros((seq_len - t, feat_dim), dtype=sequence.dtype)
                padded = np.vstack([partial, padding])
            else:
                padded = partial

            with torch.no_grad():
                X_input = torch.FloatTensor(padded).unsqueeze(0).to(device)  # (1, seq_len, feat_dim)
                out = model(X_input)
                probs = torch.softmax(out, dim=1)
                pred = int(torch.argmax(probs, dim=1).item())
                conf = float(probs[0, pred].item())

            if pred == true_label and conf >= confidence_threshold:
                detection_time = t
                detection_conf = conf
                break

        detection_times.append(int(detection_time))
        detection_confidences.append(float(detection_conf))

    return detection_times, detection_confidences


def analyze_detection_metrics(detection_times):
    valid = [t for t in detection_times if t > 0]
    never = sum(1 for t in detection_times if t <= 0)

    return {
        "total_sequences": int(len(detection_times)),
        "detected": int(len(valid)),
        "never_detected": int(never),
        "detection_rate": float(len(valid) / len(detection_times) * 100) if len(detection_times) else 0.0,
        "mean_detection_time": float(np.mean(valid)) if valid else 0.0,
        "median_detection_time": float(np.median(valid)) if valid else 0.0,
        "min_detection_time": float(np.min(valid)) if valid else 0.0,
        "max_detection_time": float(np.max(valid)) if valid else 0.0,
        "std_detection_time": float(np.std(valid)) if valid else 0.0,
    }


# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_detection_time_distribution(detection_times, model_name, output_path):
    valid_times = [t for t in detection_times if t > 0]
    if not valid_times:
        print(f"No valid detections for {model_name}")
        return

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    max_time = max(valid_times)
    bins = range(1, max_time + 2)

    axes[0].hist(valid_times, bins=bins, edgecolor="black", alpha=0.7)
    axes[0].set_xlabel("Detection Time (timesteps)")
    axes[0].set_ylabel("Frequency")
    axes[0].set_title(f"{model_name} - Distribution of Detection Times")
    axes[0].grid(True, alpha=0.3)

    sorted_times = sorted(valid_times)
    cumulative = np.arange(1, len(sorted_times) + 1) / len(detection_times) * 100

    axes[1].plot(sorted_times, cumulative, marker="o", markersize=4, linewidth=2)
    axes[1].set_xlabel("Detection Time (timesteps)")
    axes[1].set_ylabel("Cumulative Detection Rate (%)")
    axes[1].set_title(f"{model_name} - Cumulative Detection Curve")
    axes[1].grid(True, alpha=0.3)
    axes[1].axhline(y=50, linestyle="--", alpha=0.7, label="50%")
    axes[1].axhline(y=90, linestyle="--", alpha=0.7, label="90%")
    axes[1].legend()

    plt.tight_layout()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Detection distribution saved to {output_path}")


def plot_detection_by_platform(detection_times, y_true, platform_classes, model_name, output_path):
    platform_times = defaultdict(list)

    for t, label in zip(detection_times, y_true):
        if t > 0:
            platform_times[platform_classes[int(label)]].append(int(t))

    if not platform_times:
        print(f"No valid detections for {model_name}")
        return

    fig, ax = plt.subplots(figsize=(14, 8))
    platforms = list(platform_times.keys())
    times_by_platform = [platform_times[p] for p in platforms]

    ax.boxplot(times_by_platform, labels=platforms, patch_artist=True)
    ax.set_xlabel("Platform", fontsize=12, fontweight="bold")
    ax.set_ylabel("Detection Time (timesteps)", fontsize=12, fontweight="bold")
    ax.set_title(f"{model_name} - Detection Time by Breach Platform", fontsize=14, fontweight="bold")
    ax.grid(True, alpha=0.3, axis="y")
    plt.xticks(rotation=45, ha="right")

    plt.tight_layout()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Detection by platform saved to {output_path}")


# ============================================================================
# MAIN
# ============================================================================

def main():
    print("=" * 70)
    print("CROSS-PLATFORM TIME-TO-DETECTION ANALYSIS")
    print("=" * 70)

    OUTPUT_DIR = "/home/Passwords/Analysis/Platform_Output"
    DATA_PATH = "/home/Passwords/Analysis/platforms.npz"

    STAGE3_DIR = os.path.join(OUTPUT_DIR, "stage3")
    DELAY_DIR = os.path.join(STAGE3_DIR, "delay")
    os.makedirs(DELAY_DIR, exist_ok=True)

    X, y_platform, platform_classes = load_dataset(DATA_PATH)
    input_size = X.shape[2]
    num_classes = len(platform_classes)

    # NOTE: Your training script may have saved IFCNN as "ifcnn_tpp_model.pth"
    # We will try a few common filenames automatically for IFCNN-TPP.
    def resolve_model_path(primary_path: str, fallbacks: list[str]) -> str:
        if os.path.exists(primary_path):
            return primary_path
        for p in fallbacks:
            if os.path.exists(p):
                return p
        return primary_path  # return primary even if missing; caller will handle

    models_to_analyze = {
        "LSTM":      f"{OUTPUT_DIR}/LSTM_model.pth",
        "GRU":       f"{OUTPUT_DIR}/GRU_model.pth",
        "FTA-GRU":   f"{OUTPUT_DIR}/FTA-GRU_model.pth",
        "FTA-LSTM":  f"{OUTPUT_DIR}/FTA-LSTM_model.pth",
        "GNN":       f"{OUTPUT_DIR}/GNN_model.pth",
        "IFCNN-TPP": resolve_model_path(
            f"{OUTPUT_DIR}/IFCNN-TPP_model.pth",
            [
                f"{OUTPUT_DIR}/ifcnn_tpp_model.pth",
                f"{OUTPUT_DIR}/IFCNN_TPP_model.pth",
                f"{OUTPUT_DIR}/IFCNNTPP_model.pth",
            ],
        ),
        "CEP3":      f"{OUTPUT_DIR}/CEP3_model.pth",
    }

    all_stats = []

    for model_name, model_path in models_to_analyze.items():
        print(f"\n{'=' * 70}")
        print(f"Analyzing {model_name} Model")
        print(f"{'=' * 70}")

        try:
            if not os.path.exists(model_path):
                print(f"Model file not found: {model_path}")
                print("Please train the models first using the training script")
                continue

            model = load_trained_model(model_name, input_size, num_classes, model_path)

            print(f"\nComputing detection times for {model_name}...")
            detection_times, confidences = compute_detection_time(
                model, X, y_platform, confidence_threshold=0.7
            )

            stats = analyze_detection_metrics(detection_times)

            print("\nDetection Statistics:")
            print(f"  Total sequences: {stats['total_sequences']}")
            print(f"  Detected: {stats['detected']} ({stats['detection_rate']:.2f}%)")
            print(f"  Never detected: {stats['never_detected']}")
            print("\nDetection Time (timesteps):")
            print(f"  Mean: {stats['mean_detection_time']:.2f}")
            print(f"  Median: {stats['median_detection_time']:.2f}")
            print(f"  Std: {stats['std_detection_time']:.2f}")
            print(f"  Range: [{stats['min_detection_time']:.0f}, {stats['max_detection_time']:.0f}]")

            # Stage3-compatible JSON
            delay_json = {
                "model": model_name,
                "setting": "cross_platform",
                "threshold": 0.7,
                "n_rows": int(len(X)),
                "n_entities_detected": int(stats["detected"]),
                "n_entities_with_positive": int(stats["total_sequences"]),
                "detection_rate_entities": float(stats["detection_rate"] / 100.0),
                "delay_mean": float(stats["mean_detection_time"]),
                "delay_median": float(stats["median_detection_time"]),
                "delay_min": float(stats["min_detection_time"]),
                "delay_max": float(stats["max_detection_time"]),
            }

            delay_json_path = os.path.join(
                DELAY_DIR,
                f"cross_platform__{model_name.lower().replace('-', '_')}.json"
            )
            with open(delay_json_path, "w") as f:
                json.dump(delay_json, f, indent=2)
            print(f"Saved delay metrics to: {delay_json_path}")

            plot_detection_time_distribution(
                detection_times, model_name,
                f"{OUTPUT_DIR}/{model_name}_detection_distribution.png"
            )

            plot_detection_by_platform(
                detection_times, y_platform, platform_classes, model_name,
                f"{OUTPUT_DIR}/{model_name}_detection_by_platform.png"
            )

            all_stats.append({
                "Model": model_name,
                "Detection Rate (%)": stats["detection_rate"],
                "Mean Time (timesteps)": stats["mean_detection_time"],
                "Median Time (timesteps)": stats["median_detection_time"],
            })

        except Exception as e:
            print(f"Error analyzing {model_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue

    if all_stats:
        print("\n" + "=" * 70)
        print("MODEL COMPARISON - TIME-TO-DETECTION")
        print("=" * 70)

        comparison_df = pd.DataFrame(all_stats).sort_values("Detection Rate (%)", ascending=False)
        print("\n" + comparison_df.to_string(index=False))
        comparison_df.to_csv(f"{OUTPUT_DIR}/detection_time_comparison.csv", index=False)

        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

        colors1 = plt.cm.viridis(np.linspace(0.3, 0.9, len(comparison_df)))
        axes[0].bar(comparison_df["Model"], comparison_df["Detection Rate (%)"], color=colors1)
        axes[0].set_ylabel("Detection Rate (%)", fontsize=11, fontweight="bold")
        axes[0].set_title("Detection Rate Comparison", fontsize=12, fontweight="bold")
        axes[0].grid(True, alpha=0.3, axis="y")
        axes[0].tick_params(axis="x", rotation=45)

        for i, (_, row) in enumerate(comparison_df.iterrows()):
            axes[0].text(
                i, row["Detection Rate (%)"],
                f'{row["Detection Rate (%)"]:.1f}%',
                ha="center", va="bottom", fontweight="bold"
            )

        colors2 = plt.cm.plasma(np.linspace(0.3, 0.9, len(comparison_df)))
        axes[1].bar(comparison_df["Model"], comparison_df["Mean Time (timesteps)"], color=colors2)
        axes[1].set_ylabel("Mean Detection Time (timesteps)", fontsize=11, fontweight="bold")
        axes[1].set_title("Mean Detection Time Comparison", fontsize=12, fontweight="bold")
        axes[1].grid(True, alpha=0.3, axis="y")
        axes[1].tick_params(axis="x", rotation=45)

        for i, (_, row) in enumerate(comparison_df.iterrows()):
            axes[1].text(
                i, row["Mean Time (timesteps)"],
                f'{row["Mean Time (timesteps)"]:.1f}',
                ha="center", va="bottom", fontweight="bold"
            )

        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/model_detection_comparison.png", dpi=300, bbox_inches="tight")
        plt.close()
        print(f"\nComparison plot saved to {OUTPUT_DIR}/model_detection_comparison.png")

    print("\n" + "=" * 70)
    print("ANALYSIS COMPLETE!")
    print("=" * 70)
    print("\nGenerated files:")
    print("  - *_detection_distribution.png: Detection time distributions")
    print("  - *_detection_by_platform.png: Detection times by platform")
    print("  - detection_time_comparison.csv: Model comparison table")
    print("  - model_detection_comparison.png: Visual comparison")
    print("  - stage3/delay/*.json: Detection metrics (Stage3 compatible)")
    print("\nModels analyzed: LSTM, GRU, FTA-GRU, FTA-LSTM, GNN, IFCNN-TPP, CEP3 (cross-platform)")


if __name__ == "__main__":
    main()


In [1]:
import json
import math
import pandas as pd

# -------------------------------------------------------------------
# Hardcoded file paths (edit if your notebook path differs)
# -------------------------------------------------------------------
FILES = {
    "cep3":      "/home/afam/Passwords/Analysis/Detection_Output/per_platform__cep3/results_per_platform_cep3.json",
    "gru":       "/home/afam/Passwords/Analysis/Detection_Output/per_platform__fta_gru/results_per_platform_fta_gru.json",
    "fta_lstm":  "/home/afam/Passwords/Analysis/Detection_Output/per_platform__fta_lstm/results_per_platform_fta_lstm.json",
    "ifcnn_tpp": "/home/afam/Passwords/Analysis/Detection_Output/per_platform__ifcnn_tpp/results_per_platform_ifcnn_tpp.json",
    "gnn":       "/home/afam/Passwords/Analysis/Detection_Output/per_platform__gnn/results_per_platform_gnn.json",
    "lstm":      "/home/afam/Passwords/Analysis/Detection_Output/per_platform__lstm/results_per_platform_lstm.json",
    "fta_gru" : "/home/afam/Passwords/Analysis/Detection_Output/per_platform__fta_gru/results_per_platform_fta_gru.json"
}

METRICS = ["accuracy", "precision", "recall", "f1", "auroc", "auprc"]

def _is_num(x):
    return isinstance(x, (int, float))

def _is_nan(x):
    return isinstance(x, float) and math.isnan(x)

def collect_values(per_platform_metrics: dict, metric: str):
    """Collect numeric values for metric, excluding NaNs."""
    vals = []
    for _, d in per_platform_metrics.items():
        v = d.get(metric, None)
        if v is None or (not _is_num(v)) or _is_nan(v):
            continue
        vals.append(float(v))
    return vals

def best_avg(vals):
    """Return (best=max, avg=mean)."""
    if not vals:
        return None, None, 0
    return max(vals), sum(vals) / len(vals), len(vals)

def summarize_file(path: str, model_name: str):
    with open(path, "r") as f:
        obj = json.load(f)

    ppm = obj.get("per_platform_metrics", {})
    if not isinstance(ppm, dict) or not ppm:
        raise ValueError(f"{model_name}: 'per_platform_metrics' missing/empty in {path}")

    row = {"model": model_name}

    # Best + Avg for each metric (NaNs ignored)
    for m in METRICS:
        vals = collect_values(ppm, m)
        b, a, n = best_avg(vals)
        row[f"{m}_best"] = b
        row[f"{m}_avg"] = a
        row[f"{m}_n"] = n  # how many platforms contributed to this metric (after dropping NaNs)

    # optional: how many platforms had meaningful operating point
    row["num_platforms_total"] = len(ppm)
    row["num_platforms_f1_nonzero"] = sum(
        1 for _, d in ppm.items()
        if _is_num(d.get("f1", 0.0)) and float(d.get("f1", 0.0)) != 0.0
    )

    return row

# -------------------------------------------------------------------
# Build combined summary table for all models
# -------------------------------------------------------------------
rows = []
for model, path in FILES.items():
    rows.append(summarize_file(path, model))

df = pd.DataFrame(rows)

# nicer column ordering
ordered_cols = ["model", "num_platforms_total", "num_platforms_f1_nonzero"]
for m in METRICS:
    ordered_cols += [f"{m}_best", f"{m}_avg", f"{m}_n"]

df = df[ordered_cols]

# format floats for display (keep full precision in df if you want)
df_display = df.copy()
for c in df_display.columns:
    if c.endswith("_best") or c.endswith("_avg"):
        df_display[c] = df_display[c].map(lambda x: f"{x:.6f}" if isinstance(x, (int, float)) and x is not None else str(x))

df_display
df.to_csv("/home/afam/Passwords/Analysis/Detection_Output/output.csv", index=False)


In [12]:
import json
from pathlib import Path
import pandas as pd
import math

# ------------------------------------------------------------
# CONFIG: folder containing per_platform__*.json files
# ------------------------------------------------------------
BASE_DIR = Path("/home/afam/Passwords/Analysis/Detection_Output/detection_delay")   # change if needed

TARGET_MODELS = {
    "cep3",
    "gru",
    "fta_lstm",
    "ifcnn_tpp",
    "gnn",
    "lstm",
    "fta_gru",
}

# ------------------------------------------------------------
# Helper
# ------------------------------------------------------------
def safe_float(x):
    if x is None:
        return None
    if isinstance(x, float) and math.isnan(x):
        return None
    return float(x)

# ------------------------------------------------------------
# Collect detection_rate_entities per model
# ------------------------------------------------------------
records = []

for json_file in BASE_DIR.glob("per_platform__*.json"):
    name = json_file.stem
    # expected format: per_platform__MODEL__PLATFORM
    parts = name.split("__")
    if len(parts) < 3:
        continue

    _, model, platform = parts[0], parts[1], "__".join(parts[2:])
    if model not in TARGET_MODELS:
        continue

    with open(json_file, "r") as f:
        data = json.load(f)

    dr = safe_float(data.get("detection_rate_entities"))

    records.append({
        "model": model,
        "platform": platform,
        "detection_rate_entities": dr,
        "is_zero": (dr == 0.0),
    })

df = pd.DataFrame(records)

# ------------------------------------------------------------
# Aggregate: average + zero-count per model
# ------------------------------------------------------------
summary = (
    df.groupby("model")
      .agg(
          avg_detection_rate_entities=(
              "detection_rate_entities",
              lambda x: x.dropna().mean()
          ),
          zero_detection_rate_count=(
              "is_zero",
              "sum"
          ),
          num_platforms=("platform", "count")
      )
      .reset_index()
      .sort_values("model")
)

# Pretty formatting for display
summary_display = summary.copy()
summary_display["avg_detection_rate_entities"] = (
    summary_display["avg_detection_rate_entities"]
    .map(lambda x: f"{x:.6f}" if pd.notna(x) else "NaN")
)

summary_display


,model,avg_detection_rate_entities,zero_detection_rate_count,num_platforms
0,cep3,0.032990,19,22
1,fta_gru,0.017190,18,22
2,fta_lstm,0.045312,19,22
3,gnn,0.080985,18,22
4,gru,0.060258,17,22
5,ifcnn_tpp,0.000000,20,22
6,lstm,0.032046,18,22


In [2]:
import json
from pathlib import Path
import pandas as pd
import math

# ------------------------------------------------------------
# CONFIG: folder containing per_platform__*.json files
# ------------------------------------------------------------
BASE_DIR = Path("/home/afam/Passwords/Analysis/Detection_Output/detection_delay")

TARGET_MODELS = {
    "cep3",
    "gru",
    "fta_lstm",
    "ifcnn_tpp",
    "gnn",
    "lstm",
    "fta_gru",
}

# ------------------------------------------------------------
# Helper
# ------------------------------------------------------------
def safe_float(x):
    if x is None:
        return None
    if isinstance(x, float) and math.isnan(x):
        return None
    return float(x)

# ------------------------------------------------------------
# Collect detection_rate_entities per model
# ------------------------------------------------------------
records = []

for json_file in BASE_DIR.glob("per_platform__*.json"):
    name = json_file.stem
    # expected format: per_platform__MODEL__PLATFORM
    parts = name.split("__")
    if len(parts) < 3:
        continue

    _, model, platform = parts[0], parts[1], "__".join(parts[2:])
    if model not in TARGET_MODELS:
        continue

    with open(json_file, "r") as f:
        data = json.load(f)

    dr = safe_float(data.get("detection_rate_entities"))

    records.append({
        "model": model,
        "platform": platform,
        "detection_rate_entities": dr,
        "is_zero": (dr == 0.0),
    })

df = pd.DataFrame(records)

# ------------------------------------------------------------
# Aggregate: HIGHEST (not average) + zero-count per model
# ------------------------------------------------------------
summary = (
    df.groupby("model")
      .agg(
          highest_detection_rate_entities=(
              "detection_rate_entities",
              lambda x: x.dropna().max()
          ),
          zero_detection_rate_count=(
              "is_zero",
              "sum"
          ),
          num_platforms=("platform", "count")
      )
      .reset_index()
      .sort_values("model")
)

# ------------------------------------------------------------
# Pretty formatting for display
# ------------------------------------------------------------
summary_display = summary.copy()
summary_display["highest_detection_rate_entities"] = (
    summary_display["highest_detection_rate_entities"]
    .map(lambda x: f"{x:.6f}" if pd.notna(x) else "NaN")
)

summary_display


,model,highest_detection_rate_entities,zero_detection_rate_count,num_platforms
0,cep3,0.659794,19,22
1,fta_gru,0.320000,18,22
2,fta_lstm,0.906250,19,22
3,gnn,0.989691,18,22
4,gru,0.660000,17,22
5,ifcnn_tpp,0.000000,20,22
6,lstm,0.610000,18,22
